In [6]:
%pwd

'/Users/ganeshprasadbhandari/Documents/D_drive/clark/careeragent-ai/notebooks_v2'

In [7]:
%cd ..

/Users/ganeshprasadbhandari/Documents/D_drive/clark/careeragent-ai


In [8]:
%pwd

'/Users/ganeshprasadbhandari/Documents/D_drive/clark/careeragent-ai'

In [2]:
from pathlib import Path
import os

# notebooks_v2 -> repo root
os.chdir(Path.cwd().parent)
print("CWD =", Path.cwd())
assert (Path.cwd() / "src").exists(), "Still not at repo root"

CWD = /Users/ganeshprasadbhandari/Documents/D_drive/clark/careeragent-ai


In [3]:
# NOTEBOOK CELL — FULL ONE-CLICK AUTONOMY UPGRADE (Engine + Dashboard)
# Overwrites:
# - src/careeragent/orchestration/engine.py
# - src/careeragent/orchestration/__init__.py
# - app/ui/dashboard.py  (adds F1/OPT sponsorship toggle + recency + max jobs)
# - app/main.py          (robust bootstrap)

from __future__ import annotations
from pathlib import Path
from datetime import datetime

ROOT = Path.cwd().resolve()
assert (ROOT / "src").exists(), f"Run from repo root. CWD={ROOT}"

def backup_write(rel_path: str, content: str) -> None:
    p = ROOT / rel_path
    p.parent.mkdir(parents=True, exist_ok=True)
    if p.exists():
        ts = datetime.now().strftime("%Y%m%d_%H%M%S")
        bak = p.with_suffix(p.suffix + f".bak_{ts}")
        bak.write_text(p.read_text(encoding="utf-8"), encoding="utf-8")
        print("BACKUP:", bak)
    p.write_text(content, encoding="utf-8")
    print("WROTE:", p)

# ---------------------------
# 1) Engine (full L0-L9 loop)
# ---------------------------
ENGINE_PY = r'''
from __future__ import annotations

import json
import re
import threading
from collections import Counter, defaultdict
from dataclasses import dataclass
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple
from uuid import uuid4

import httpx

from careeragent.config import artifacts_root, get_settings
from careeragent.orchestration.state import OrchestrationState
from careeragent.services.db_service import SqliteStateStore
from careeragent.services.notification_service import NotificationService
from careeragent.services.health_service import HealthService

from careeragent.agents.security_agent import SanitizeAgent

# Optional (if present in your repo from previous batches)
try:
    from careeragent.agents.guardrail_service import OutputGuard  # type: ignore
except Exception:
    OutputGuard = None  # type: ignore

from careeragent.agents.parser_agent_service import ParserAgentService, ExtractedResume
from careeragent.agents.parser_evaluator_service import ParserEvaluatorService

# If your existing matcher services exist, we use them. Otherwise we use local scoring.
try:
    from careeragent.agents.matcher_agent_schema import JobDescription  # type: ignore
    from careeragent.agents.matcher_agent_service import MatcherAgentService  # type: ignore
except Exception:
    JobDescription = None  # type: ignore
    MatcherAgentService = None  # type: ignore

# Draft/apply/XAI are best-effort if available
try:
    from careeragent.agents.cover_letter_service import CoverLetterService  # type: ignore
    from careeragent.agents.cover_letter_evaluator_service import CoverLetterEvaluatorService  # type: ignore
except Exception:
    CoverLetterService = None  # type: ignore
    CoverLetterEvaluatorService = None  # type: ignore

try:
    from careeragent.agents.strategy_agent_service import StrategyAgentService  # type: ignore
except Exception:
    StrategyAgentService = None  # type: ignore

try:
    from careeragent.agents.apply_executor_service import ApplyExecutorService  # type: ignore
except Exception:
    ApplyExecutorService = None  # type: ignore

try:
    from careeragent.services.xai_service import XAIService  # type: ignore
    from careeragent.services.exporter import CareerDossierExporter  # type: ignore
except Exception:
    XAIService = None  # type: ignore
    CareerDossierExporter = None  # type: ignore


@dataclass(frozen=True)
class JobBoard:
    name: str
    domain: str


DEFAULT_JOB_BOARDS: Tuple[JobBoard, ...] = (
    JobBoard("LinkedIn Jobs", "linkedin.com/jobs"),
    JobBoard("Indeed", "indeed.com"),
    JobBoard("Glassdoor", "glassdoor.com"),
    JobBoard("ZipRecruiter", "ziprecruiter.com"),
    JobBoard("Monster", "monster.com"),
    JobBoard("Dice", "dice.com"),
    JobBoard("Lever", "jobs.lever.co"),
    JobBoard("Greenhouse", "boards.greenhouse.io"),
)


VISA_NEGATIVE = (
    "unable to sponsor",
    "cannot sponsor",
    "no sponsorship",
    "do not sponsor",
    "not sponsor",
    "without sponsorship",
    "must be authorized to work",
    "no visa",
    "cannot provide visa",
)

VISA_POSITIVE = (
    "visa sponsorship",
    "sponsorship available",
    "h1b",
    "opt",
    "cpt",
    "stem opt",
    "work visa",
)

COMMON_SKILL_DICTIONARY = [
    "python","sql","fastapi","docker","kubernetes","mlflow","dvc","azure","aws","gcp",
    "pytorch","tensorflow","sklearn","pandas","numpy","spark","databricks","snowflake",
    "llm","rag","langgraph","langchain","vector database","faiss","chroma","postgres","redis",
    "etl","airflow","kafka","terraform","github actions","sagemaker","azure ml","openai"
]


def _run_dir(run_id: str) -> Path:
    d = artifacts_root() / "runs" / run_id
    d.mkdir(parents=True, exist_ok=True)
    return d


def _save_json(path: Path, obj: Any) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    path.write_text(json.dumps(obj, indent=2), encoding="utf-8")


class LiveFeed:
    """
    Description: Live Agent Feed logger (UI consumes state.meta['live_feed']).
    Layer: L1
    """
    @staticmethod
    def emit(state: OrchestrationState, *, layer: str, agent: str, message: str) -> None:
        state.meta.setdefault("live_feed", [])
        state.meta["live_feed"].append({"layer": layer, "agent": agent, "message": message})
        state.touch()


class LocalResumeExtractor:
    """
    Description: Extract resume text from PDF/TXT/DOCX bytes.
    Layer: L2
    """
    @staticmethod
    def extract_text(*, filename: str, data: bytes) -> str:
        name = (filename or "").lower()
        if name.endswith(".txt"):
            return data.decode("utf-8", errors="replace")

        if name.endswith(".pdf"):
            try:
                from pypdf import PdfReader  # type: ignore
            except Exception as e:
                raise RuntimeError("PDF extraction needs pypdf. Install: uv add pypdf") from e
            import io
            reader = PdfReader(io.BytesIO(data))
            return "\n".join([(pg.extract_text() or "") for pg in reader.pages])

        if name.endswith(".docx"):
            try:
                import docx  # type: ignore
            except Exception as e:
                raise RuntimeError("DOCX extraction needs python-docx. Install: uv add python-docx") from e
            import io
            f = io.BytesIO(data)
            d = docx.Document(f)
            return "\n".join([p.text for p in d.paragraphs if p.text])

        # fallback
        return data.decode("utf-8", errors="replace")


class OllamaClient:
    """
    Description: Optional local LLM used for query refinement + job summaries.
    Layer: L4-L5
    """
    def __init__(self, base_url: Optional[str], model: str) -> None:
        self._base = (base_url or "").strip().rstrip("/")
        self._model = model

    def available(self) -> bool:
        return bool(self._base)

    def generate(self, *, prompt: str, timeout: float = 45.0) -> str:
        if not self.available():
            return ""
        try:
            url = self._base + "/api/generate"
            payload = {"model": self._model, "prompt": prompt, "stream": False}
            with httpx.Client(timeout=timeout) as client:
                r = client.post(url, json=payload)
            if r.status_code >= 400:
                return ""
            return (r.json().get("response") or "").strip()
        except Exception:
            return ""


class SerperDiscoveryService:
    """
    Description: Serper discovery across job boards with best-effort recency.
    Layer: L3
    """
    SERPER_URL = "https://google.serper.dev/search"

    def __init__(self, *, api_key: str, health: HealthService) -> None:
        self._key = api_key
        self._health = health

    def search(self, *, state: OrchestrationState, step_id: str, query: str, num: int = 20, tbs: Optional[str] = None) -> List[Dict[str, Any]]:
        headers = {"X-API-KEY": self._key, "Content-Type": "application/json"}
        body: Dict[str, Any] = {"q": query, "num": num}
        if tbs:
            # Serper may accept tbs; if it doesn't, it will just ignore it (best-effort)
            body["tbs"] = tbs

        try:
            with httpx.Client(timeout=30.0) as client:
                r = client.post(self.SERPER_URL, headers=headers, json=body)
        except Exception as e:
            state.status = "api_failure"
            state.meta["run_failure_code"] = "API_FAILURE"
            state.meta["run_failure_provider"] = "serper"
            LiveFeed.emit(state, layer="L3", agent="DiscoveryService", message=f"Serper request failed: {e}")
            return []

        if self._health.quota.handle_serper_response(
            state=state, step_id=step_id, status_code=r.status_code, tool_name="serper.search", error_detail=r.text[:200]
        ):
            return []

        if r.status_code >= 400:
            state.status = "api_failure"
            state.meta["run_failure_code"] = "API_FAILURE"
            state.meta["run_failure_provider"] = "serper"
            LiveFeed.emit(state, layer="L3", agent="DiscoveryService", message=f"Serper error {r.status_code}: {r.text[:200]}")
            return []

        organic = (r.json().get("organic") or [])
        out = []
        for it in organic:
            out.append({"title": it.get("title") or "", "link": it.get("link") or "", "snippet": it.get("snippet") or ""})
        return out


class ScraperService:
    """
    Description: Scrape job pages best-effort. Falls back to snippet.
    Layer: L3-L4
    """
    @staticmethod
    def fetch_text(*, url: str, snippet: str) -> str:
        if not url:
            return snippet or ""
        try:
            with httpx.Client(timeout=18.0, follow_redirects=True) as client:
                r = client.get(url, headers={"User-Agent": "Mozilla/5.0"})
            if r.status_code >= 400:
                return snippet or ""
            html = r.text
            txt = re.sub(r"<(script|style)[^>]*>.*?</\1>", " ", html, flags=re.S | re.I)
            txt = re.sub(r"<[^>]+>", " ", txt)
            txt = re.sub(r"\s+", " ", txt).strip()
            if not txt:
                return snippet or ""
            return txt[:16000]
        except Exception:
            return snippet or ""


def _tokenize(text: str) -> List[str]:
    return re.findall(r"[a-zA-Z][a-zA-Z0-9\+\#\.-]{1,}", (text or "").lower())


def cosine_sim(a: Counter, b: Counter) -> float:
    if not a or not b:
        return 0.0
    dot = sum(a[t] * b.get(t, 0) for t in a)
    na = sum(v*v for v in a.values()) ** 0.5
    nb = sum(v*v for v in b.values()) ** 0.5
    if na == 0 or nb == 0:
        return 0.0
    return float(dot / (na * nb))


def parse_recency_hours(text: str) -> Optional[float]:
    """
    Best-effort parsing from snippets like "12 hours ago", "1 day ago", "yesterday", "today".
    """
    s = (text or "").lower()
    if "today" in s:
        return 6.0
    if "yesterday" in s:
        return 24.0
    m = re.search(r"(\d+)\s*(hour|hours)\s*ago", s)
    if m:
        return float(m.group(1))
    m = re.search(r"(\d+)\s*(day|days)\s*ago", s)
    if m:
        return float(m.group(1)) * 24.0
    m = re.search(r"(\d+)\s*(minute|minutes)\s*ago", s)
    if m:
        return max(1.0, float(m.group(1)) / 60.0)
    return None


def ats_score_from_text(resume_text: str) -> float:
    """
    Lightweight ATS heuristic for the resume itself (0-1).
    """
    t = resume_text.lower()
    score = 0.0
    # contact signals
    if re.search(r"[\w\.-]+@[\w\.-]+\.\w+", t):
        score += 0.18
    if re.search(r"\+?\d[\d\-\s\(\)]{8,}\d", t):
        score += 0.10
    # headings
    for h in ["summary", "experience", "education", "skills", "projects"]:
        if h in t:
            score += 0.12
    # bullets / structure
    if "-" in resume_text or "•" in resume_text:
        score += 0.12
    # length (not too short)
    if len(resume_text) > 1500:
        score += 0.12
    return max(0.0, min(1.0, score))


def visa_ok(job_text: str) -> bool:
    low = (job_text or "").lower()
    return not any(x in low for x in VISA_NEGATIVE)


def visa_positive(job_text: str) -> bool:
    low = (job_text or "").lower()
    return any(x in low for x in VISA_POSITIVE)


def extract_job_skills(job_text: str, resume_skills: List[str]) -> List[str]:
    low = (job_text or "").lower()
    skills = []
    pool = list(dict.fromkeys([*(resume_skills or []), *COMMON_SKILL_DICTIONARY]))
    for s in pool:
        s2 = str(s).lower().strip()
        if not s2:
            continue
        if s2 in low:
            skills.append(s2)
    return list(dict.fromkeys(skills))[:30]


def compute_interview_chance(*, skill_overlap: float, exp_align: float, ats: float, market: float) -> float:
    market = max(1.0, float(market))
    score = (0.45 * skill_overlap + 0.35 * exp_align + 0.20 * ats) / market
    return max(0.0, min(1.0, float(score)))


class DiscoveryEvaluatorAgent:
    """
    Description: L5 evaluator that gates discovery quality and triggers query refinement.
    Layer: L5
    """
    def evaluate(self, *, top_score: float, avg_score: float, n: int, threshold: float) -> Tuple[float, List[str]]:
        fb: List[str] = []
        # Confidence model: top weighted more than avg.
        conf = min(1.0, (0.65 * top_score) + (0.35 * avg_score))
        if n < 10:
            conf -= 0.10
            fb.append("Too few jobs after filters. Broaden search terms or expand boards.")
        if top_score < threshold:
            conf -= 0.15
            fb.append(f"Top match below threshold ({top_score*100:.1f}% < {threshold*100:.0f}%). Refining query.")
        conf = max(0.0, min(1.0, conf))
        return conf, fb


class OneClickAutomationEngine:
    """
    Description: CareerOS-style One-Click autonomous workflow with HITL gates.
    Layer: L0-L9
    """

    def __init__(self) -> None:
        self._s = get_settings()
        self._store = SqliteStateStore()

        self._health = HealthService()
        self._health.load_env(dotenv_path=str(Path(".env")))
        self._health.enable_langsmith_tracing(project=self._s.langsmith_project)

        self._notifier = NotificationService(dry_run=not bool(self._s.twilio_account_sid))
        self._sanitize = SanitizeAgent()

        self._ollama = OllamaClient(self._s.ollama_base_url, getattr(self._s, "ollama_model", "llama3.2"))

        self._parser = ParserAgentService()
        self._parser_eval = ParserEvaluatorService()

        self._matcher = MatcherAgentService() if MatcherAgentService else None
        self._disc_eval = DiscoveryEvaluatorAgent()

        self._out_guard = OutputGuard() if OutputGuard else None
        self._cover = CoverLetterService() if CoverLetterService else None
        self._cover_eval = CoverLetterEvaluatorService() if CoverLetterEvaluatorService else None
        self._strategy = StrategyAgentService() if StrategyAgentService else None
        self._apply = ApplyExecutorService() if ApplyExecutorService else None

    def _persist(self, state: OrchestrationState) -> None:
        d = _run_dir(state.run_id)
        _save_json(d / "state.json", state.model_dump())
        self._store.upsert_state(run_id=state.run_id, status=state.status, state=state.model_dump(), updated_at_utc=state.updated_at_utc)

    def load(self, run_id: str) -> Optional[Dict[str, Any]]:
        return self._store.get_state(run_id=run_id)

    def start_run(self, *, filename: str, data: bytes, prefs: Dict[str, Any]) -> OrchestrationState:
        st = OrchestrationState.new(env=self._s.environment, mode="agentic", git_sha=None)
        st.meta["preferences"] = prefs
        st.meta.setdefault("job_scores", {})
        st.meta.setdefault("job_components", {})
        st.meta.setdefault("job_meta", {})
        LiveFeed.emit(st, layer="L1", agent="Dashboard", message="Run created. Starting autonomous pipeline…")
        self._persist(st)

        t = threading.Thread(target=self._run_pipeline, args=(st.run_id, filename, data), daemon=True)
        t.start()
        return st

    def submit_action(self, *, run_id: str, action_type: str, payload: Dict[str, Any]) -> OrchestrationState:
        raw = self.load(run_id)
        if not raw:
            raise ValueError("run_id not found")
        st = OrchestrationState(**raw)
        st.meta["last_user_action"] = {"type": action_type, "payload": payload}
        LiveFeed.emit(st, layer="L5", agent="HITL", message=f"User action received: {action_type}")
        self._persist(st)

        t = threading.Thread(target=self._continue_after_hitl, args=(run_id,), daemon=True)
        t.start()
        return st

    # ---------------- PIPELINE ----------------
    def _run_pipeline(self, run_id: str, filename: str, data: bytes) -> None:
        raw = self.load(run_id)
        if not raw:
            return
        st = OrchestrationState(**raw)
        run_dir = _run_dir(run_id)

        prefs = st.meta.get("preferences", {}) or {}
        recency_hours = float(prefs.get("recency_hours", 36))
        max_jobs = int(prefs.get("max_jobs", 40))
        target_role = str(prefs.get("target_role", "Data Scientist"))
        location = str(prefs.get("location", "United States"))
        remote = bool(prefs.get("remote", True))
        wfo_ok = bool(prefs.get("wfo_ok", True))
        salary = str(prefs.get("salary", "")).strip()
        visa_required = bool(prefs.get("visa_sponsorship_required", False))
        threshold = float(prefs.get("discovery_threshold", 0.70))
        max_refinements = int(prefs.get("max_refinements", 3))

        # L2 extract resume text
        st.start_step("l2_extract", layer_id="L2", tool_name="ResumeExtractor", input_ref={"filename": filename})
        LiveFeed.emit(st, layer="L2", agent="ParserAgent", message="Extracting resume text from upload…")
        try:
            resume_text = LocalResumeExtractor.extract_text(filename=filename, data=data)
        except Exception as e:
            st.end_step("l2_extract", status="failed", output_ref={}, message=str(e))
            st.status = "needs_human_approval"
            st.meta["pending_action"] = "resume_extract_failed"
            LiveFeed.emit(st, layer="L2", agent="ParserAgent", message=f"Resume extraction failed: {e}")
            self._persist(st)
            return

        (run_dir / "resume_raw.txt").write_text(resume_text, encoding="utf-8")
        st.add_artifact("resume_raw", str(run_dir / "resume_raw.txt"), content_type="text/plain")
        st.end_step("l2_extract", status="ok", output_ref={"artifact_key": "resume_raw"}, message="extracted")
        self._persist(st)

        # L0 sanitize
        st.start_step("l0_sanitize", layer_id="L0", tool_name="SanitizeAgent", input_ref={})
        safe = self._sanitize.sanitize_before_llm(
            state=st, step_id="l0_sanitize", tool_name="sanitize_before_llm",
            user_text=resume_text, context="resume"
        )
        if safe is None:
            LiveFeed.emit(st, layer="L0", agent="SanitizeAgent", message="Prompt injection detected. Run blocked.")
            self._persist(st)
            return
        st.end_step("l0_sanitize", status="ok", output_ref={"sanitized": True}, message="pass")
        LiveFeed.emit(st, layer="L0", agent="SanitizeAgent", message="Security check passed.")
        self._persist(st)

        # L2/L3 parse recursive gate
        extracted = self._parse_with_gate(st=st, safe_text=safe, run_dir=run_dir)
        self._persist(st)
        if st.status == "needs_human_approval":
            self._notify(st, event="needs_human_approval")
            return

        resume_skills = [str(s).lower() for s in (extracted.skills or []) if s]
        ats = ats_score_from_text(safe)
        st.meta["ats_score"] = ats
        LiveFeed.emit(st, layer="L2", agent="ParserAgent", message=f"Profile extracted. Skills={len(resume_skills)} ATS={ats:.2f}")

        if not self._s.serper_api_key:
            st.status = "needs_human_approval"
            st.meta["pending_action"] = "missing_serper_key"
            LiveFeed.emit(st, layer="L3", agent="DiscoveryAgent", message="Missing SERPER_API_KEY in .env.")
            self._persist(st)
            return

        discovery = SerperDiscoveryService(api_key=self._s.serper_api_key, health=self._health)

        # ----- L3-L5 discovery refinement loop -----
        base_query = self._build_query(target_role=target_role, location=location, remote=remote, wfo_ok=wfo_ok, salary=salary, visa_required=visa_required, skills=resume_skills)
        tbs = "qdr:d" if recency_hours <= 36 else None

        all_ranked: List[Dict[str, Any]] = []
        for attempt in range(max_refinements + 1):
            LiveFeed.emit(st, layer="L3", agent="DiscoveryAgent", message=f"Hunt attempt {attempt+1}: searching 8 job boards…")
            st.start_step(f"l3_discovery_{attempt+1}", layer_id="L3", tool_name="DiscoveryService", input_ref={"query": base_query, "tbs": tbs})

            results, board_counts = self._search_boards(st=st, discovery=discovery, base_query=base_query, tbs=tbs)
            st.end_step(f"l3_discovery_{attempt+1}", status="ok", output_ref={"results": len(results), "boards": board_counts}, message="discovered")
            self._persist(st)

            if st.status in ("blocked", "api_failure"):
                self._notify(st, event="quota_error", extra={"provider": "serper"})
                return

            # L4 scrape + score
            LiveFeed.emit(st, layer="L4", agent="ScraperAgent", message=f"Scraping + normalizing up to {max_jobs} jobs…")
            ranked = self._scrape_and_score(
                st=st,
                results=results[:max_jobs],
                extracted=extracted,
                resume_text=safe,
                resume_skills=resume_skills,
                ats=ats,
                recency_hours=recency_hours,
                visa_required=visa_required,
            )
            all_ranked = ranked
            _save_json(run_dir / "ranking.json", ranked)
            st.add_artifact("ranking", str(run_dir / "ranking.json"), content_type="application/json")
            _save_json(run_dir / "search_summary.json", {"query": base_query, "tbs": tbs, "boards": board_counts, "raw_results": len(results), "kept": len(ranked)})
            st.add_artifact("search_summary", str(run_dir / "search_summary.json"), content_type="application/json")
            self._persist(st)

            if not ranked:
                conf, fb = 0.0, ["No jobs survived filters. Widen recency or disable visa-required filter."]
            else:
                top = float(ranked[0]["interview_chance_score"])
                avg = sum(float(x["interview_chance_score"]) for x in ranked[:20]) / max(1, min(20, len(ranked)))
                conf, fb = self._disc_eval.evaluate(top_score=top, avg_score=avg, n=len(ranked), threshold=threshold)

            LiveFeed.emit(st, layer="L5", agent="EvaluatorAgent", message=f"Discovery confidence={conf:.2f} (threshold={threshold:.2f}).")
            if fb:
                LiveFeed.emit(st, layer="L5", agent="EvaluatorAgent", message=" | ".join(fb[:2]))

            # Pass -> HITL review ranking
            if ranked and float(ranked[0]["interview_chance_score"]) >= threshold:
                st.status = "needs_human_approval"
                st.meta["pending_action"] = "review_ranking"
                LiveFeed.emit(st, layer="L1", agent="Dashboard", message="Ranking ready for review (HITL).")
                self._persist(st)
                self._notify(st, event="needs_human_approval")
                return

            # Low confidence after final attempt -> HITL
            if attempt >= max_refinements:
                st.status = "needs_human_approval"
                st.meta["pending_action"] = "low_confidence_discovery"
                LiveFeed.emit(st, layer="L5", agent="EvaluatorAgent", message="Low confidence after retries. Needs human guidance.")
                self._persist(st)
                self._notify(st, event="needs_human_approval")
                return

            # refine query
            base_query = self._refine_query(
                st=st,
                base_query=base_query,
                ranked=ranked,
                resume_skills=resume_skills,
                target_role=target_role,
                location=location,
                visa_required=visa_required,
            )
            LiveFeed.emit(st, layer="L3", agent="DiscoveryAgent", message=f"Refining query → {base_query[:160]}")
            self._persist(st)

    def _continue_after_hitl(self, run_id: str) -> None:
        raw = self.load(run_id)
        if not raw:
            return
        st = OrchestrationState(**raw)
        run_dir = _run_dir(run_id)

        pending = st.meta.get("pending_action")
        action = (st.meta.get("last_user_action") or {}).get("type")
        payload = (st.meta.get("last_user_action") or {}).get("payload") or {}

        # Reject ranking => restart discovery with extra notes
        if pending == "review_ranking" and action == "reject_ranking":
            st.status = "running"
            st.meta["pending_action"] = None
            st.meta.setdefault("user_refinement_notes", [])
            st.meta["user_refinement_notes"].append(payload.get("reason", "User rejected ranking"))
            LiveFeed.emit(st, layer="L5", agent="HITL", message="Ranking rejected. Re-running discovery with refined intent…")
            self._persist(st)

            # Resume text + extracted should exist already; just rerun pipeline discovery section by restarting whole run is heavy.
            # Simple approach: re-run full pipeline with the already uploaded resume.
            resume_path = run_dir / "resume_raw.txt"
            if not resume_path.exists():
                st.status = "needs_human_approval"
                st.meta["pending_action"] = "resume_missing"
                self._persist(st)
                return
            resume_text = resume_path.read_text(encoding="utf-8")
            # Fake a file name; we only need text. We'll re-enter pipeline by calling _run_pipeline with same text via TXT bytes.
            self._run_pipeline(run_id, "resume.txt", resume_text.encode("utf-8"))
            return

        # Approve ranking => generate drafts (L6) then pause for draft review
        if pending == "review_ranking" and action == "approve_ranking":
            LiveFeed.emit(st, layer="L6", agent="Generator", message="Ranking approved. Generating strategy + cover letters…")
            st.status = "running"
            st.meta["pending_action"] = None
            self._persist(st)

            ranking_path = run_dir / "ranking.json"
            if not ranking_path.exists():
                st.status = "needs_human_approval"
                st.meta["pending_action"] = "ranking_missing"
                self._persist(st)
                return

            ranked = json.loads(ranking_path.read_text(encoding="utf-8"))
            top_n = int((st.meta.get("preferences", {}) or {}).get("draft_count", 10))
            ranked = ranked[:top_n]

            # Load extracted resume
            ex_ref = st.artifacts.get("extracted_resume")
            if not ex_ref or not Path(ex_ref.path).exists():
                st.status = "needs_human_approval"
                st.meta["pending_action"] = "extracted_resume_missing"
                self._persist(st)
                return
            extracted = ExtractedResume(**json.loads(Path(ex_ref.path).read_text(encoding="utf-8")))

            drafts_bundle = self._generate_drafts(st=st, extracted=extracted, ranked=ranked, run_dir=run_dir)
            _save_json(run_dir / "drafts.json", drafts_bundle)
            st.add_artifact("drafts_bundle", str(run_dir / "drafts.json"), content_type="application/json")

            st.status = "needs_human_approval"
            st.meta["pending_action"] = "review_drafts"
            LiveFeed.emit(st, layer="L1", agent="Dashboard", message="Drafts ready for review (HITL).")
            self._persist(st)
            self._notify(st, event="needs_human_approval")
            return

        # Approve drafts => apply + XAI + dossier => completed
        if pending == "review_drafts" and action == "approve_drafts":
            LiveFeed.emit(st, layer="L7", agent="ApplyExecutor", message="Drafts approved. Submitting (simulated)…")
            st.status = "running"
            st.meta["pending_action"] = None
            self._persist(st)

            ok = self._apply_and_finalize(st=st, run_dir=run_dir)
            if ok:
                st.status = "completed"
                LiveFeed.emit(st, layer="L9", agent="Analytics", message="Run completed. Dossier ready.")
                self._persist(st)
                self._notify(st, event="completed")
            else:
                st.status = "needs_human_approval"
                st.meta["pending_action"] = "apply_failed"
                self._persist(st)
                self._notify(st, event="needs_human_approval")
            return

        # Reject drafts => back to ranking review
        if pending == "review_drafts" and action == "reject_drafts":
            st.status = "needs_human_approval"
            st.meta["pending_action"] = "review_ranking"
            st.meta.setdefault("user_refinement_notes", [])
            st.meta["user_refinement_notes"].append(payload.get("reason", "User rejected drafts"))
            LiveFeed.emit(st, layer="L5", agent="HITL", message="Drafts rejected. Returning to ranking review.")
            self._persist(st)
            self._notify(st, event="needs_human_approval")
            return

    # ---------------- helpers ----------------
    def _notify(self, st: OrchestrationState, *, event: str, extra: Optional[Dict[str, Any]] = None) -> None:
        prefs = st.meta.get("preferences", {}) or {}
        to = prefs.get("user_phone") or getattr(self._s, "user_phone", None)
        if to:
            self._notifier.notify(state=st, event=event, extra=extra, to_phone=str(to))

    def _parse_with_gate(self, *, st: OrchestrationState, safe_text: str, run_dir: Path) -> ExtractedResume:
        fb: List[str] = []
        extracted: Optional[ExtractedResume] = None

        for attempt in range(4):
            sid = f"l2_parse_{attempt+1}"
            st.start_step(sid, layer_id="L2", tool_name="ParserAgentService", input_ref={"attempt": attempt+1})
            LiveFeed.emit(st, layer="L2", agent="ParserAgent", message=f"Parsing resume (attempt {attempt+1})…")
            extracted = self._parser.parse(raw_text=safe_text, orchestration_state=st, feedback=fb)
            p = run_dir / f"extracted_resume_attempt_{attempt+1}.json"
            _save_json(p, extracted.to_json_dict())
            st.add_artifact(f"extracted_resume_attempt_{attempt+1}", str(p), content_type="application/json")
            st.end_step(sid, status="ok", output_ref={"artifact_key": f"extracted_resume_attempt_{attempt+1}"}, message="parsed")

            ev = self._parser_eval.evaluate(
                orchestration_state=st, raw_text=safe_text, extracted=extracted,
                target_id="resume_main", threshold=0.80, retry_count=attempt, max_retries=3
            )
            decision = st.apply_recursive_gate(target_id="resume_main", layer_id="L3")
            LiveFeed.emit(st, layer="L3", agent="ParserEvaluator", message=f"Parse quality={ev.evaluation_score:.2f} decision={decision}")
            self._persist(st)

            if decision == "pass":
                st.add_artifact("extracted_resume", str(p), content_type="application/json")
                return extracted

            if decision == "human_approval":
                st.status = "needs_human_approval"
                st.meta["pending_action"] = "resume_cleanup"
                return extracted

            fb = ev.feedback

        st.status = "needs_human_approval"
        st.meta["pending_action"] = "resume_cleanup"
        return extracted or ExtractedResume()

    def _build_query(self, *, target_role: str, location: str, remote: bool, wfo_ok: bool, salary: str, visa_required: bool, skills: List[str]) -> str:
        intent = []
        if remote:
            intent.append("remote")
        if wfo_ok:
            intent.append("hybrid")
        if not intent:
            intent.append("on-site")
        intent_str = " ".join(intent)

        skill_str = " ".join(skills[:6])
        salary_part = f'"{salary}"' if salary else ""
        visa_part = '"visa sponsorship" OR h1b OR opt OR cpt' if visa_required else ""
        # Keep it Google-friendly
        return f'{target_role} {location} {intent_str} {salary_part} {skill_str} ({visa_part}) apply'

    def _search_boards(self, *, st: OrchestrationState, discovery: SerperDiscoveryService, base_query: str, tbs: Optional[str]) -> Tuple[List[Dict[str, Any]], Dict[str, int]]:
        seen = set()
        out: List[Dict[str, Any]] = []
        counts: Dict[str, int] = {}
        for b in DEFAULT_JOB_BOARDS:
            q = f'{base_query} site:{b.domain}'
            step_id = f"l3_serper_{b.domain.replace('/','_')}"
            st.start_step(step_id, layer_id="L3", tool_name="serper.search", input_ref={"board": b.name})
            items = discovery.search(state=st, step_id=step_id, query=q, num=10, tbs=tbs)
            st.end_step(step_id, status="ok", output_ref={"count": len(items)}, message=b.name)
            counts[b.name] = len(items)

            for it in items:
                link = it.get("link") or ""
                if not link or link in seen:
                    continue
                seen.add(link)
                it["board"] = b.name
                out.append(it)

        LiveFeed.emit(st, layer="L3", agent="DiscoveryAgent", message=f"Found {len(out)} unique roles across boards.")
        return out, counts

    def _scrape_and_score(
        self,
        *,
        st: OrchestrationState,
        results: List[Dict[str, Any]],
        extracted: ExtractedResume,
        resume_text: str,
        resume_skills: List[str],
        ats: float,
        recency_hours: float,
        visa_required: bool,
    ) -> List[Dict[str, Any]]:
        ranked: List[Dict[str, Any]] = []

        res_tokens = Counter(_tokenize(resume_text))
        exp_text = " ".join([str(x) for x in (extracted.experience or [])])[:5000]
        exp_tokens = Counter(_tokenize(exp_text)) if exp_text else res_tokens

        # decide per-job
        for idx, it in enumerate(results):
            url = it.get("link") or ""
            title = it.get("title") or ""
            snippet = it.get("snippet") or ""
            board = it.get("board") or "unknown"

            # recency filter
            rh = parse_recency_hours(snippet)
            if rh is not None and rh > recency_hours:
                continue

            step_id = f"l4_scrape_{idx+1}"
            st.start_step(step_id, layer_id="L4", tool_name="ScraperService", input_ref={"url": url, "board": board})
            job_text = ScraperService.fetch_text(url=url, snippet=snippet)
            st.end_step(step_id, status="ok", output_ref={"chars": len(job_text)}, message="scraped")

            # visa filter
            v_ok = visa_ok(job_text)
            if visa_required and not v_ok:
                continue

            job_skills = extract_job_skills(job_text, resume_skills)
            if job_skills:
                overlap = len(set(job_skills) & set(resume_skills)) / max(1, len(set(job_skills)))
            else:
                overlap = 0.0

            job_tokens = Counter(_tokenize(job_text))
            exp_align = cosine_sim(exp_tokens, job_tokens)

            market = 1.0
            # very rough market factor heuristic from snippet
            low = snippet.lower()
            if "applicants" in low:
                m = re.search(r"(\d+)\+?\s*applicants", low)
                if m:
                    n = int(m.group(1))
                    market = 1.0 + min(1.5, n / 200.0)  # 200 applicants => 2.0

            score = compute_interview_chance(skill_overlap=overlap, exp_align=exp_align, ats=ats, market=market)

            # boost if visa-positive signals and visa required
            if visa_required and visa_positive(job_text):
                score = min(1.0, score + 0.05)

            rationale = [
                f"SkillOverlap={overlap:.2f}",
                f"ExperienceAlignment={exp_align:.2f}",
                f"ATS={ats:.2f}",
                f"MarketFactor={market:.2f}",
                ("VisaOK" if v_ok else "NoSponsorship"),
            ]

            ranked.append(
                {
                    "rank": 0,
                    "title": title,
                    "company": board,
                    "board": board,
                    "url": url,
                    "recency_hours": rh,
                    "visa_ok": v_ok,
                    "matched_skills": list(set(job_skills) & set(resume_skills))[:12],
                    "missing_skills": [s for s in job_skills if s not in resume_skills][:12],
                    "skill_overlap": overlap,
                    "experience_alignment": exp_align,
                    "ats_score": ats,
                    "market_factor": market,
                    "interview_chance_score": score,
                    "overall_match_percent": round(score * 100.0, 2),
                    "rationale": rationale,
                }
            )

            # meta for UI/XAI
            jid = url or str(uuid4().hex)
            st.meta.setdefault("job_scores", {})
            st.meta.setdefault("job_components", {})
            st.meta.setdefault("job_meta", {})
            st.meta["job_scores"][jid] = float(score)
            st.meta["job_components"][jid] = {
                "skill_overlap": float(overlap),
                "experience_alignment": float(exp_align),
                "ats_score": float(ats),
                "market_competition_factor": float(market),
            }
            st.meta["job_meta"][jid] = {"role_title": title, "company": board, "url": url, "source": board}

        ranked.sort(key=lambda x: float(x["interview_chance_score"]), reverse=True)
        for i, r in enumerate(ranked, start=1):
            r["rank"] = i

        LiveFeed.emit(st, layer="L4", agent="MatcherAgent", message=f"Scored {len(ranked)} jobs. Top score={ranked[0]['overall_match_percent'] if ranked else 'n/a'}")
        return ranked

    def _refine_query(
        self,
        *,
        st: OrchestrationState,
        base_query: str,
        ranked: List[Dict[str, Any]],
        resume_skills: List[str],
        target_role: str,
        location: str,
        visa_required: bool,
    ) -> str:
        top = ranked[0] if ranked else {}
        top_sk = (top.get("matched_skills") or [])[:6]
        skill_hint = " ".join(list(dict.fromkeys([*top_sk, *resume_skills]))[:8])

        # include user refinement notes if any
        notes = st.meta.get("user_refinement_notes", []) or []
        notes_hint = " ".join([str(n)[:30] for n in notes[-2:]]) if notes else ""

        if self._ollama.available():
            prompt = (
                "You refine job search queries.\n"
                f"Target role: {target_role}\nLocation: {location}\n"
                f"Current query: {base_query}\n"
                f"Skill hint: {skill_hint}\n"
                f"Visa required: {visa_required}\n"
                f"User notes: {notes_hint}\n"
                "Return ONE concise improved query string only."
            )
            safe = self._sanitize.sanitize_before_llm(state=st, step_id="l0_query_refine", tool_name="sanitize_before_llm", user_text=prompt, context="chat")
            if safe:
                out = self._ollama.generate(prompt=safe, timeout=35.0)
                if out:
                    return out.replace("\n", " ")[:220]

        visa_part = '("visa sponsorship" OR h1b OR opt)' if visa_required else ""
        return f"{target_role} {location} {skill_hint} {visa_part} apply"

    def _generate_drafts(self, *, st: OrchestrationState, extracted: ExtractedResume, ranked: List[Dict[str, Any]], run_dir: Path) -> Dict[str, Any]:
        drafts: List[Dict[str, Any]] = []
        strategies: List[Dict[str, Any]] = []

        # Simple deterministic fallback if CoverLetterService missing
        def fallback_cover(job_title: str, company: str) -> str:
            email = getattr(getattr(extracted, "contact", None), "email", "") or ""
            name = getattr(extracted, "name", "Candidate")
            return f"""{name}
{email}

Dear Hiring Manager,

I’m applying for the {job_title} role. I bring strong experience in Python, ML/GenAI delivery, and production MLOps. I focus on shipping reliable systems, not demos—clean APIs, reproducible pipelines, and measurable business impact.

I’d welcome the chance to discuss how I can help {company} deliver practical AI outcomes.

Sincerely,
{name}
"""

        for j in ranked:
            title = j.get("title") or j.get("role_title") or "Role"
            company = j.get("company") or j.get("board") or "Company"
            url = j.get("url") or ""

            # Strategy (best effort)
            if self._strategy:
                try:
                    # If you have a full MatchReport in your repo, wire it here later.
                    # For now: strategy is lightweight.
                    strategies.append({"job": title, "company": company, "action_items": ["Emphasize top matched skills", "Quantify impact", "Align summary to role"]})
                except Exception as e:
                    strategies.append({"job": title, "company": company, "error": str(e)})
            else:
                strategies.append({"job": title, "company": company, "action_items": ["Emphasize top matched skills", "Quantify impact", "Align summary to role"]})

            # Cover letter
            body = ""
            eval_score = None
            guard_action = None
            guard_issues: List[str] = []

            if self._cover and self._cover_eval and JobDescription and self._matcher:
                try:
                    # Create a minimal JobDescription for generation
                    jd = JobDescription(
                        job_id=str(uuid4().hex),
                        role_title=title,
                        company=company,
                        country_code=str((st.meta.get("preferences", {}) or {}).get("country", "US")),
                        required_skills=[],
                        preferred_skills=[],
                        responsibilities=[],
                        requirements_text="",
                        applicants_count=None,
                        market_competition_factor=None,
                    )
                    # Minimal match report substitute: reuse job rationale fields for context
                    # If your CoverLetterService expects MatchReport, it will still work if implemented that way in your repo.
                    draft = self._cover.draft(resume=extracted, job=jd, match_report=None, orchestration_state=st, feedback=["Include contact header"])  # type: ignore
                    body = draft.body
                    # evaluator twin (best effort)
                    ev = self._cover_eval.evaluate(orchestration_state=st, resume=extracted, job=jd, match_report=None, draft=draft, target_id=f"cover::{title}", threshold=0.80, retry_count=0, max_retries=3)  # type: ignore
                    eval_score = float(getattr(ev, "evaluation_score", 0.0))
                except Exception:
                    body = fallback_cover(title, company)
            else:
                body = fallback_cover(title, company)

            # Output guard
            if self._out_guard:
                try:
                    gr = self._out_guard.check_cover_letter(state=st, draft_text=body, resume=extracted, match_report=None, country_code=str((st.meta.get("preferences", {}) or {}).get("country", "US")))  # type: ignore
                    guard_action = gr.action
                    guard_issues = gr.issues[:3]
                except Exception:
                    pass

            p = run_dir / f"cover_letter_{re.sub(r'[^a-zA-Z0-9]+','_',title)[:40]}.md"
            p.write_text(body, encoding="utf-8")
            st.add_artifact(p.stem, str(p), content_type="text/markdown")

            drafts.append({
                "title": title,
                "company": company,
                "url": url,
                "cover_path": str(p),
                "evaluator_score": eval_score,
                "guard_action": guard_action,
                "guard_issues": guard_issues,
            })

        LiveFeed.emit(st, layer="L6", agent="CoverLetterAgent", message=f"Generated {len(drafts)} cover letters.")
        return {"drafts": drafts, "strategies": strategies}

    def _apply_and_finalize(self, *, st: OrchestrationState, run_dir: Path) -> bool:
        # Apply simulation (if ApplyExecutorService exists)
        if not self._apply:
            LiveFeed.emit(st, layer="L7", agent="ApplyExecutor", message="Apply executor missing. Skipping apply.")
        else:
            # Submit top 3 (configurable)
            ranking_path = run_dir / "ranking.json"
            if ranking_path.exists():
                ranked = json.loads(ranking_path.read_text(encoding="utf-8"))[:3]
                subs: List[Dict[str, Any]] = []
                for j in ranked:
                    try:
                        job_id = str(j.get("url") or uuid4().hex)
                        # pick any cover artifact key
                        # (we stored markdown files with stems; for simplicity, apply uses placeholders here)
                        sub = self._apply.submit(orchestration_state=st, job_id=job_id, resume_artifact_key="extracted_resume", cover_letter_artifact_key="drafts_bundle", notes="Simulated apply")  # type: ignore
                        subs.append(sub.model_dump())
                    except Exception:
                        continue
                _save_json(run_dir / "submissions.json", {"submissions": subs})
                st.add_artifact("submissions", str(run_dir / "submissions.json"), content_type="application/json")

        # XAI + dossier
        if XAIService and CareerDossierExporter:
            try:
                xai = XAIService()
                xai_paths = xai.write_outputs(state=st, require_reportlab=False)
                st.add_artifact("xai_transparency_pdf", xai_paths["pdf"], content_type="application/pdf")
                st.add_artifact("transparency_matrix_json", xai_paths["json"], content_type="application/json")

                exporter = CareerDossierExporter()
                bundle = exporter.bundle_reports(run_id=st.run_id, final_pdf_path=xai_paths["pdf"])
                st.add_artifact("career_dossier_zip", bundle["zip"], content_type="application/zip")
            except Exception as e:
                LiveFeed.emit(st, layer="L9", agent="Analytics", message=f"XAI/Dossier generation failed: {e}")
                return False

        return True


ENGINE = OneClickAutomationEngine()
'''
backup_write("src/careeragent/orchestration/engine.py", ENGINE_PY)

INIT_PY = r'''
from __future__ import annotations

from .state import OrchestrationState  # noqa: F401
from .engine import OneClickAutomationEngine, ENGINE  # noqa: F401
'''
backup_write("src/careeragent/orchestration/__init__.py", INIT_PY)

# ---------------------------
# 2) Dashboard (add F1/OPT toggle + recency + max jobs)
# ---------------------------
DASHBOARD = r'''
from __future__ import annotations

import json
import os
import sys
from pathlib import Path
from typing import Any, Dict, Optional

import requests
import streamlit as st

REPO_ROOT = Path(__file__).resolve().parents[2]
SRC = REPO_ROOT / "src"
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

DEFAULT_API = os.getenv("API_URL", "http://127.0.0.1:8000")


def _safe_json(resp: requests.Response) -> Dict[str, Any]:
    try:
        return resp.json()
    except Exception:
        return {"_raw": resp.text[:1500], "_status_code": resp.status_code}


def _api_get(api_base: str, path: str, timeout: int = 20) -> requests.Response:
    return requests.get(f"{api_base}{path}", timeout=timeout)


def _api_post(api_base: str, path: str, timeout: int = 60, **kwargs) -> requests.Response:
    return requests.post(f"{api_base}{path}", timeout=timeout, **kwargs)


def _exists(path: Optional[str]) -> bool:
    return bool(path) and Path(path).exists()


def _download(label: str, path: str, mime: str) -> None:
    p = Path(path)
    if not p.exists():
        st.warning(f"Missing file: {path}")
        return
    st.download_button(label, data=p.read_bytes(), file_name=p.name, mime=mime, use_container_width=True)


def main() -> None:
    st.set_page_config(page_title="CareerAgent-AI Mission Control", layout="wide")
    st.title("CareerAgent-AI — Mission Control (One-Click Automation)")
    st.caption("Upload resume → autonomous ingestion + discovery + ranking → HITL approvals → drafts + dossier downloads")

    api_base = st.sidebar.text_input("API Base URL", value=DEFAULT_API)

    st.sidebar.divider()
    st.sidebar.subheader("Backend")
    try:
        h = _api_get(api_base, "/health", timeout=3)
        if h.status_code == 200:
            st.sidebar.success("🟢 API Online")
        else:
            st.sidebar.warning(f"🟠 API issue ({h.status_code})")
    except Exception as e:
        st.sidebar.error("🔴 API Offline")
        st.sidebar.caption(str(e))
        st.stop()

    st.sidebar.divider()
    st.sidebar.subheader("Resume Upload")
    resume_file = st.sidebar.file_uploader("Upload Resume (PDF/TXT/DOCX)", type=["pdf", "txt", "docx"])

    st.sidebar.subheader("Preferences")
    target_role = st.sidebar.text_input("Target role", value="Data Scientist")
    country = st.sidebar.text_input("Country", value="US")
    location = st.sidebar.text_input("Location", value="United States")

    remote = st.sidebar.checkbox("Remote preferred", value=True)
    wfo_ok = st.sidebar.checkbox("On-site/WFO acceptable", value=True)

    salary = st.sidebar.text_input("Salary target (optional)", value="")
    visa_required = st.sidebar.checkbox("Visa sponsorship required (F1/OPT)", value=False)

    recency_hours = st.sidebar.slider("Only jobs posted within (hours)", 12, 168, 36, 6)
    max_jobs = st.sidebar.slider("Max jobs to score per run", 10, 80, 40, 5)

    user_phone = st.sidebar.text_input("Phone for SMS (optional)", value="")

    st.sidebar.subheader("Autonomy Controls")
    discovery_threshold = st.sidebar.slider("Discovery confidence threshold", 0.50, 0.90, 0.70, 0.05)
    max_refinements = st.sidebar.slider("Max query refinements", 1, 6, 3, 1)
    draft_count = st.sidebar.slider("Draft count (top jobs)", 3, 25, 10, 1)

    run_btn = st.sidebar.button("🚀 RUN ONE-CLICK", type="primary", use_container_width=True, disabled=(resume_file is None))

    st.sidebar.divider()
    st.sidebar.subheader("Existing Run")
    run_id_in = st.sidebar.text_input("Run ID", value=st.session_state.get("run_id", ""))

    if run_btn:
        prefs = {
            "target_role": target_role.strip() or "Data Scientist",
            "country": country.strip() or "US",
            "location": location.strip() or "United States",
            "remote": bool(remote),
            "wfo_ok": bool(wfo_ok),
            "salary": salary.strip(),
            "visa_sponsorship_required": bool(visa_required),
            "recency_hours": float(recency_hours),
            "max_jobs": int(max_jobs),
            "discovery_threshold": float(discovery_threshold),
            "max_refinements": int(max_refinements),
            "draft_count": int(draft_count),
            "user_phone": user_phone.strip() or None,
        }

        files = {"resume": (resume_file.name, resume_file.getvalue())}
        data = {"preferences_json": json.dumps(prefs)}
        r = _api_post(api_base, "/analyze", files=files, data=data, timeout=180)

        if r.status_code >= 400:
            st.error(f"/analyze failed: {r.status_code}\n\n{r.text[:1500]}")
            st.stop()

        out = _safe_json(r)
        st.session_state["run_id"] = out["run_id"]
        st.success(f"Run started: {out['run_id']} (status: {out.get('status')})")

    run_id = st.session_state.get("run_id") or run_id_in.strip()
    if not run_id:
        st.info("Upload a resume and click RUN, or paste an existing run_id.")
        return

    colA, colB, colC = st.columns([1, 1, 1])
    _ = st.button("🔄 Refresh", use_container_width=True)

    r = _api_get(api_base, f"/status/{run_id}", timeout=25)
    if r.status_code != 200:
        st.warning(f"Run not found yet ({r.status_code}).")
        return

    state = _safe_json(r)
    status = state.get("status", "unknown")
    meta = state.get("meta", {}) or {}
    feed = meta.get("live_feed", []) or []
    steps = state.get("steps", []) or []
    artifacts = state.get("artifacts", {}) or {}
    pending = meta.get("pending_action")

    with colA:
        st.metric("Run ID", run_id)
    with colB:
        st.metric("Status", status)
    with colC:
        st.metric("Pending", str(pending))

    total = max(1, len(steps))
    done = sum(1 for s in steps if s.get("finished_at_utc"))
    st.progress(done / total)

    left, right = st.columns([1.25, 0.75], vertical_alignment="top")

    with left:
        st.markdown("### Live Agent Feed")
        if not feed:
            st.info("No live feed yet.")
        else:
            for ev in feed[-160:]:
                st.write(f"**[{ev.get('layer')} {ev.get('agent')}]** {ev.get('message')}")

        with st.expander("Audit Trail (Steps)"):
            for s in steps:
                st.write(f"- [{s.get('layer_id')}] {s.get('tool_name')} | {s.get('status')} | {s.get('step_id')}")

    with right:
        st.markdown("### Key Scores")
        # Show top score if ranking exists
        ranking_ref = artifacts.get("ranking")
        if ranking_ref and _exists(ranking_ref.get("path")):
            ranking = json.loads(Path(ranking_ref["path"]).read_text(encoding="utf-8"))
            if ranking:
                st.metric("Top Match", f"{ranking[0]['overall_match_percent']}%")
        else:
            st.caption("Scores not available yet.")

        st.markdown("### Downloads")
        zip_ref = artifacts.get("career_dossier_zip")
        pdf_ref = artifacts.get("xai_transparency_pdf")
        if zip_ref and _exists(zip_ref.get("path")):
            _download("Download Career Dossier (ZIP)", zip_ref["path"], "application/zip")
        else:
            st.caption("Dossier ZIP not ready.")
        if pdf_ref and _exists(pdf_ref.get("path")):
            _download("Download XAI Transparency (PDF)", pdf_ref["path"], "application/pdf")
        else:
            st.caption("XAI PDF not ready.")

    st.markdown("### Ranking")
    ranking_ref = artifacts.get("ranking")
    if ranking_ref and _exists(ranking_ref.get("path")):
        ranking = json.loads(Path(ranking_ref["path"]).read_text(encoding="utf-8"))
        st.dataframe(
            [{
                "rank": x.get("rank"),
                "score_%": x.get("overall_match_percent"),
                "title": x.get("title"),
                "board": x.get("board"),
                "recency_h": x.get("recency_hours"),
                "visa_ok": x.get("visa_ok"),
                "url": x.get("url"),
                "missing_skills": ", ".join((x.get("missing_skills") or [])[:6]),
            } for x in ranking],
            use_container_width=True
        )
    else:
        st.caption("Ranking not available yet.")

    st.markdown("### Human-in-the-Loop Controls")
    if status == "needs_human_approval" and pending == "review_ranking":
        c1, c2 = st.columns(2)
        with c1:
            if st.button("✅ Approve Ranking → Generate Drafts", type="primary", use_container_width=True):
                _api_post(api_base, f"/action/{run_id}", json={"action_type": "approve_ranking", "payload": {}}, timeout=30)
                st.success("Approved. Draft generation started.")
        with c2:
            reason = st.text_input("Reason to refine ranking", value="")
            if st.button("❌ Reject Ranking → Re-run Discovery", use_container_width=True):
                _api_post(api_base, f"/action/{run_id}", json={"action_type": "reject_ranking", "payload": {"reason": reason}}, timeout=30)
                st.warning("Rejected. Discovery refinement started.")

    if status == "needs_human_approval" and pending == "review_drafts":
        c1, c2 = st.columns(2)
        with c1:
            if st.button("✅ Approve Drafts → Submit (Simulated)", type="primary", use_container_width=True):
                _api_post(api_base, f"/action/{run_id}", json={"action_type": "approve_drafts", "payload": {}}, timeout=30)
                st.success("Approved. Submission started.")
        with c2:
            reason = st.text_input("Reason to revise drafts", value="")
            if st.button("❌ Reject Drafts → Back to Ranking", use_container_width=True):
                _api_post(api_base, f"/action/{run_id}", json={"action_type": "reject_drafts", "payload": {"reason": reason}}, timeout=30)
                st.warning("Rejected drafts. Returning to ranking review.")

    st.markdown("### Drafts Bundle (preview)")
    drafts_ref = artifacts.get("drafts_bundle")
    if drafts_ref and _exists(drafts_ref.get("path")):
        bundle = json.loads(Path(drafts_ref["path"]).read_text(encoding="utf-8"))
        st.json(bundle)
    else:
        st.caption("Draft bundle not available yet.")

    with st.expander("Full State JSON"):
        st.json(state)


if __name__ == "__main__":
    main()
'''
backup_write("app/ui/dashboard.py", DASHBOARD)

# ---------------------------
# 3) app/main.py (robust)
# ---------------------------
APP_MAIN = r'''
from __future__ import annotations

import sys
from pathlib import Path

ROOT = Path(__file__).resolve().parents[1]
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))
SRC = ROOT / "src"
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

from app.ui.dashboard import main
main()
'''
backup_write("app/main.py", APP_MAIN)

print("\n✅ FULL AUTONOMY ENGINE + UPDATED DASHBOARD WRITTEN.")
print("Next:")
print("1) Restart backend")
print("   PYTHONPATH=src uv run python -m uvicorn careeragent.api.main:app --host 127.0.0.1 --port 8000 --reload")
print("2) Restart frontend")
print("   API_URL=http://127.0.0.1:8000 PYTHONPATH='.:src' uv run streamlit run app/main.py --server.port 8501")

BACKUP: /Users/ganeshprasadbhandari/Documents/D_drive/clark/careeragent-ai/src/careeragent/orchestration/engine.py.bak_20260221_004939
WROTE: /Users/ganeshprasadbhandari/Documents/D_drive/clark/careeragent-ai/src/careeragent/orchestration/engine.py
BACKUP: /Users/ganeshprasadbhandari/Documents/D_drive/clark/careeragent-ai/src/careeragent/orchestration/__init__.py.bak_20260221_004939
WROTE: /Users/ganeshprasadbhandari/Documents/D_drive/clark/careeragent-ai/src/careeragent/orchestration/__init__.py
BACKUP: /Users/ganeshprasadbhandari/Documents/D_drive/clark/careeragent-ai/app/ui/dashboard.py.bak_20260221_004939
WROTE: /Users/ganeshprasadbhandari/Documents/D_drive/clark/careeragent-ai/app/ui/dashboard.py
BACKUP: /Users/ganeshprasadbhandari/Documents/D_drive/clark/careeragent-ai/app/main.py.bak_20260221_004939
WROTE: /Users/ganeshprasadbhandari/Documents/D_drive/clark/careeragent-ai/app/main.py

✅ FULL AUTONOMY ENGINE + UPDATED DASHBOARD WRITTEN.
Next:
1) Restart backend
   PYTHONPATH=src 

In [4]:
%pwd

'/Users/ganeshprasadbhandari/Documents/D_drive/clark/careeragent-ai'

In [5]:
from __future__ import annotations

import os
import re
import json
from pathlib import Path
from datetime import datetime

# --- auto-chdir to repo root even if notebook runs from notebooks_v2 ---
ROOT = Path.cwd().resolve()
if ROOT.name == "notebooks_v2":
    os.chdir(ROOT.parent)
ROOT = Path.cwd().resolve()
assert (ROOT / "src").exists(), f"Run from repo root. CWD={ROOT}"
print("✅ CWD =", ROOT)

def backup_write(rel_path: str, content: str) -> None:
    p = ROOT / rel_path
    p.parent.mkdir(parents=True, exist_ok=True)
    if p.exists():
        ts = datetime.now().strftime("%Y%m%d_%H%M%S")
        bak = p.with_suffix(p.suffix + f".bak_{ts}")
        bak.write_text(p.read_text(encoding="utf-8"), encoding="utf-8")
        print("BACKUP:", bak)
    p.write_text(content, encoding="utf-8")
    print("WROTE:", p)

# ---------------------------
# 1) Strong deterministic parser (CareerOS-style)
# ---------------------------
PARSER_AGENT = r'''
from __future__ import annotations

import re
from typing import Any, List, Optional

from pydantic import BaseModel, Field, ConfigDict


class ContactInfo(BaseModel):
    """
    Description: Contact info extracted from resume text.
    Layer: L2
    Input: raw resume text
    Output: structured contact object
    """
    model_config = ConfigDict(extra="ignore")
    email: Optional[str] = None
    phone: Optional[str] = None
    linkedin: Optional[str] = None
    github: Optional[str] = None
    location: Optional[str] = None


class ExperienceItem(BaseModel):
    """
    Description: Experience item extracted from resume text.
    Layer: L2
    Input: raw resume text
    Output: structured experience item
    """
    model_config = ConfigDict(extra="ignore")
    title: Optional[str] = None
    company: Optional[str] = None
    start: Optional[str] = None
    end: Optional[str] = None
    bullets: List[str] = Field(default_factory=list)


class EducationItem(BaseModel):
    """
    Description: Education item extracted from resume text.
    Layer: L2
    Input: raw resume text
    Output: structured education item
    """
    model_config = ConfigDict(extra="ignore")
    degree: Optional[str] = None
    school: Optional[str] = None
    year: Optional[str] = None


COMMON_SKILLS = [
    "python","sql","fastapi","docker","kubernetes","mlflow","dvc",
    "aws","azure","gcp","sagemaker","azure ml","databricks","spark",
    "pytorch","tensorflow","scikit-learn","pandas","numpy",
    "langchain","langgraph","rag","vector","faiss","chroma",
    "terraform","github actions","ci/cd","kafka","airflow","snowflake"
]


def _find_email(text: str) -> Optional[str]:
    m = re.search(r"[\w\.-]+@[\w\.-]+\.\w+", text or "")
    return m.group(0) if m else None


def _find_phone(text: str) -> Optional[str]:
    # handles +1 (xxx) xxx-xxxx, xxx-xxx-xxxx, etc
    m = re.search(r"(\+?\d{1,3}[\s\-]?)?(\(?\d{3}\)?[\s\-]?)\d{3}[\s\-]?\d{4}", text or "")
    return m.group(0).strip() if m else None


def _find_url(text: str, keyword: str) -> Optional[str]:
    # capture URLs containing linkedin/github
    pat = rf"(https?://[^\s]+{keyword}[^\s]*)"
    m = re.search(pat, text or "", flags=re.IGNORECASE)
    return m.group(1).strip() if m else None


def _split_sections(text: str) -> dict[str, str]:
    """
    Description: Split text into sections using heading keywords.
    Layer: L2
    Input: raw text
    Output: dict section->content
    """
    t = text or ""
    # Normalize headings
    headings = ["summary", "skills", "experience", "education", "projects", "certifications"]
    idxs = []
    low = t.lower()
    for h in headings:
        m = re.search(rf"(^|\n)\s*{h}\s*[:\-]?\s*", low)
        if m:
            idxs.append((m.start(), h))
    if not idxs:
        return {"full": t}

    idxs.sort(key=lambda x: x[0])
    out: dict[str, str] = {}
    for i, (pos, h) in enumerate(idxs):
        end = idxs[i + 1][0] if i + 1 < len(idxs) else len(t)
        out[h] = t[pos:end].strip()
    out["full"] = t
    return out


def _extract_skills(text: str) -> List[str]:
    low = (text or "").lower()
    found = []
    # 1) explicit skills section comma split
    m = re.search(r"skills\s*[:\-]\s*(.+)", text or "", flags=re.IGNORECASE)
    if m:
        chunk = m.group(1)
        parts = re.split(r"[,\|/•\n]+", chunk)
        found.extend([p.strip().lower() for p in parts if p.strip()])
    # 2) dictionary scan
    for s in COMMON_SKILLS:
        if s in low:
            found.append(s)
    # cleanup
    found = [re.sub(r"\s+", " ", x) for x in found]
    return sorted(list(dict.fromkeys([x for x in found if 2 <= len(x) <= 40])))[:50]


def _extract_name(text: str) -> Optional[str]:
    # first non-empty line that isn't a heading
    lines = [ln.strip() for ln in (text or "").splitlines() if ln.strip()]
    if not lines:
        return None
    cand = lines[0]
    if cand.lower().startswith(("summary", "skills", "experience", "education")):
        return None
    # avoid emails/phones
    if "@" in cand or re.search(r"\d{3}[\s\-]?\d{3}", cand):
        return None
    return cand[:60]


class ExtractedResume(BaseModel):
    """
    Description: Core parsed resume model for downstream layers (L3-L9).
    Layer: L2
    Input: raw resume text
    Output: structured resume JSON
    """
    model_config = ConfigDict(extra="ignore")
    name: Optional[str] = None
    contact: ContactInfo = Field(default_factory=ContactInfo)
    summary: Optional[str] = None
    skills: List[str] = Field(default_factory=list)
    experience: List[ExperienceItem] = Field(default_factory=list)
    education: List[EducationItem] = Field(default_factory=list)
    raw_text: Optional[str] = None
    warnings: List[str] = Field(default_factory=list)

    def to_json_dict(self) -> dict[str, Any]:
        return self.model_dump()


class ParserAgentService:
    """
    Description: Convert raw resume string into ExtractedResume JSON (deterministic, ATS-friendly extraction).
    Layer: L2
    Input: raw resume string
    Output: ExtractedResume
    """
    def parse(self, *, raw_text: str, orchestration_state: Any, feedback: List[str] | None = None) -> ExtractedResume:
        text = raw_text or ""
        sections = _split_sections(text)

        name = _extract_name(text)
        email = _find_email(text)
        phone = _find_phone(text)
        linkedin = _find_url(text, "linkedin")
        github = _find_url(text, "github")

        skills = _extract_skills(sections.get("skills") or text)

        summary = None
        if "summary" in sections:
            summary = re.sub(r"(?i)^summary\s*[:\-]?\s*", "", sections["summary"]).strip()
        else:
            # fallback: first 2-3 lines after name
            lines = [ln.strip() for ln in text.splitlines() if ln.strip()]
            if len(lines) >= 2:
                summary = " ".join(lines[1:3])[:400]

        # Experience: keep it simple but structured
        exp_items: List[ExperienceItem] = []
        exp_text = sections.get("experience")
        if exp_text:
            body = re.sub(r"(?i)^experience\s*[:\-]?\s*", "", exp_text).strip()
            bullets = [b.strip("-• ").strip() for b in re.split(r"\n+", body) if b.strip()]
            bullets = [b for b in bullets if len(b) > 15][:10]
            if bullets:
                exp_items.append(ExperienceItem(bullets=bullets))

        edu_items: List[EducationItem] = []
        edu_text = sections.get("education")
        if edu_text:
            body = re.sub(r"(?i)^education\s*[:\-]?\s*", "", edu_text).strip()
            # naive degree/school capture
            edu_items.append(EducationItem(degree=body[:120]))

        warnings: List[str] = []
        if not email and not phone:
            warnings.append("Missing email/phone in resume text.")
        if len(skills) < 5:
            warnings.append("Low skill extraction count; consider adding a clearer Skills section.")
        if not exp_items:
            warnings.append("Experience section not clearly detected.")

        return ExtractedResume(
            name=name,
            contact=ContactInfo(email=email, phone=phone, linkedin=linkedin, github=github),
            summary=summary,
            skills=skills,
            experience=exp_items,
            education=edu_items,
            raw_text=text,
            warnings=warnings,
        )
'''
backup_write("src/careeragent/agents/parser_agent_service.py", PARSER_AGENT)

# ---------------------------
# 2) Parser evaluator becomes soft-gate (auto-continue unless truly broken)
# ---------------------------
PARSER_EVAL = r'''
from __future__ import annotations

from typing import Any, List, Optional
from pydantic import BaseModel, Field, ConfigDict

from careeragent.agents.parser_agent_service import ExtractedResume


class EvaluationEvent(BaseModel):
    """
    Description: Evaluation output for recursive gate decisions.
    Layer: L3
    Input: generator output
    Output: score + feedback + gate decision support
    """
    model_config = ConfigDict(extra="ignore")
    evaluation_score: float = 0.0
    threshold: float = 0.0
    feedback: List[str] = Field(default_factory=list)


def _ats_structure_score(text: str) -> float:
    t = (text or "").lower()
    score = 0.0
    if "summary" in t: score += 0.15
    if "skills" in t: score += 0.20
    if "experience" in t: score += 0.25
    if "education" in t: score += 0.10
    if "-" in text or "•" in text: score += 0.15
    if len(text) > 1200: score += 0.15
    return max(0.0, min(1.0, score))


class ParserEvaluatorService:
    """
    Description: Critique Parser output for completeness + ATS readiness (soft gate).
    Layer: L3
    Input: raw_text + ExtractedResume
    Output: EvaluationEvent (score + feedback)
    """
    def evaluate(
        self,
        *,
        orchestration_state: Any,
        raw_text: str,
        extracted: ExtractedResume,
        target_id: str,
        threshold: float = 0.55,
        retry_count: int = 0,
        max_retries: int = 3,
    ) -> Any:
        feedback: List[str] = []

        contact = extracted.contact or None
        has_contact = bool(getattr(contact, "email", None) or getattr(contact, "phone", None))
        has_skills = len(extracted.skills or []) >= 6
        has_exp = bool(extracted.experience and len(extracted.experience) > 0)
        ats = _ats_structure_score(raw_text)

        # weights (so missing contact doesn't kill automation)
        w_contact = 0.15
        w_skills = 0.35
        w_exp = 0.35
        w_ats = 0.15

        s_contact = 1.0 if has_contact else 0.0
        s_skills = min(1.0, len(extracted.skills or []) / 12.0)
        s_exp = 1.0 if has_exp else 0.0
        s_ats = ats

        score = (w_contact*s_contact) + (w_skills*s_skills) + (w_exp*s_exp) + (w_ats*s_ats)
        score = max(0.0, min(1.0, float(score)))

        if not has_contact:
            feedback.append("Contact info missing (email/phone). Optional: paste improved resume to continue with stronger ATS fit.")
        if not has_exp:
            feedback.append("Experience section unclear. Add an Experience heading with bullet points.")
        if len(extracted.skills or []) < 6:
            feedback.append("Skills extraction low. Add a clearer Skills section with comma-separated skills.")
        if ats < 0.5:
            feedback.append("ATS structure weak. Add headings: Summary, Skills, Experience, Education; use bullets.")

        # Record in orchestration state (your OrchestrationState already has record_evaluation)
        ev = orchestration_state.record_evaluation(
            layer_id="L3",
            target_id=target_id,
            generator_agent="parser_agent_service",
            evaluator_agent="parser_evaluator_service",
            evaluation_score=score,
            threshold=threshold,
            feedback=feedback,
            retry_count=retry_count,
            max_retries=max_retries,
            interview_chance={
                "weights": {"w1_skill_overlap": 0.45, "w2_experience_alignment": 0.35, "w3_ats_score": 0.20},
                "components": {"skill_overlap": 0.0, "experience_alignment": 0.0, "ats_score": float(ats), "market_competition_factor": 1.0},
                "interview_chance_score": float((0.20*ats)),  # parser-stage proxy only
            },
        )
        return ev
'''
backup_write("src/careeragent/agents/parser_evaluator_service.py", PARSER_EVAL)

# ---------------------------
# 3) Engine patch: soft-gate resume_cleanup + add HITL resume submit action
#    (We keep your current full autonomy engine file name and overwrite it)
# ---------------------------
ENGINE_PATCH = r'''
from __future__ import annotations

import json
import re
import threading
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple
from uuid import uuid4

import httpx

from careeragent.config import artifacts_root, get_settings
from careeragent.orchestration.state import OrchestrationState
from careeragent.services.db_service import SqliteStateStore
from careeragent.services.notification_service import NotificationService

# HealthService is optional; if missing, we degrade gracefully
try:
    from careeragent.services.health_service import HealthService  # type: ignore
except Exception:
    HealthService = None  # type: ignore

from careeragent.agents.security_agent import SanitizeAgent
from careeragent.agents.parser_agent_service import ParserAgentService, ExtractedResume
from careeragent.agents.parser_evaluator_service import ParserEvaluatorService

# --------------------------- helpers ---------------------------
def _run_dir(run_id: str) -> Path:
    d = artifacts_root() / "runs" / run_id
    d.mkdir(parents=True, exist_ok=True)
    return d

def _save_json(path: Path, obj: Any) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    path.write_text(json.dumps(obj, indent=2), encoding="utf-8")


class LiveFeed:
    """
    Description: Live Agent Feed logger.
    Layer: L1
    """
    @staticmethod
    def emit(st: OrchestrationState, *, layer: str, agent: str, message: str) -> None:
        st.meta.setdefault("live_feed", [])
        st.meta["live_feed"].append({"layer": layer, "agent": agent, "message": message})
        st.touch()


class LocalResumeExtractor:
    """
    Description: Extract resume text from PDF/TXT/DOCX.
    Layer: L2
    """
    @staticmethod
    def extract_text(*, filename: str, data: bytes) -> str:
        name = (filename or "").lower()
        if name.endswith(".txt"):
            return data.decode("utf-8", errors="replace")

        if name.endswith(".pdf"):
            from pypdf import PdfReader  # type: ignore
            import io
            reader = PdfReader(io.BytesIO(data))
            return "\n".join([(pg.extract_text() or "") for pg in reader.pages])

        if name.endswith(".docx"):
            import docx  # type: ignore
            import io
            d = docx.Document(io.BytesIO(data))
            return "\n".join([p.text for p in d.paragraphs if p.text])

        return data.decode("utf-8", errors="replace")


# -------- Job discovery (Serper across 8 boards) --------
JOB_BOARDS = [
    ("LinkedIn Jobs", "linkedin.com/jobs"),
    ("Indeed", "indeed.com"),
    ("Glassdoor", "glassdoor.com"),
    ("ZipRecruiter", "ziprecruiter.com"),
    ("Monster", "monster.com"),
    ("Dice", "dice.com"),
    ("Lever", "jobs.lever.co"),
    ("Greenhouse", "boards.greenhouse.io"),
]

VISA_NEGATIVE = ("unable to sponsor","cannot sponsor","no sponsorship","do not sponsor","not sponsor","without sponsorship","no visa")
VISA_POSITIVE = ("visa sponsorship","h1b","opt","cpt","stem opt","work visa")


def _parse_recency_hours(snippet: str) -> Optional[float]:
    s = (snippet or "").lower()
    if "today" in s: return 6.0
    if "yesterday" in s: return 24.0
    m = re.search(r"(\d+)\s*hours?\s*ago", s)
    if m: return float(m.group(1))
    m = re.search(r"(\d+)\s*days?\s*ago", s)
    if m: return float(m.group(1)) * 24.0
    return None


def _tokenize(text: str) -> List[str]:
    return re.findall(r"[a-zA-Z][a-zA-Z0-9\+\#\.-]{1,}", (text or "").lower())


def _cosine(a: Dict[str,int], b: Dict[str,int]) -> float:
    if not a or not b: return 0.0
    dot = sum(v * b.get(k,0) for k,v in a.items())
    na = sum(v*v for v in a.values()) ** 0.5
    nb = sum(v*v for v in b.values()) ** 0.5
    if na == 0 or nb == 0: return 0.0
    return float(dot/(na*nb))


def _ats_score(resume_text: str) -> float:
    t = (resume_text or "").lower()
    score = 0.0
    if re.search(r"[\w\.-]+@[\w\.-]+\.\w+", t): score += 0.20
    if re.search(r"\+?\d[\d\-\s\(\)]{8,}\d", t): score += 0.10
    for h in ["summary","skills","experience","education","projects"]:
        if h in t: score += 0.12
    if "-" in resume_text or "•" in resume_text: score += 0.10
    if len(resume_text) > 1200: score += 0.12
    return max(0.0, min(1.0, score))


def _compute_interview_chance(skill_overlap: float, exp_align: float, ats: float, market: float) -> float:
    market = max(1.0, float(market))
    score = (0.45*skill_overlap + 0.35*exp_align + 0.20*ats) / market
    return max(0.0, min(1.0, float(score)))


class SerperClient:
    SERPER_URL = "https://google.serper.dev/search"

    def __init__(self, api_key: str) -> None:
        self.api_key = api_key

    def search(self, query: str, num: int = 10, tbs: Optional[str] = None) -> List[Dict[str, Any]]:
        headers = {"X-API-KEY": self.api_key, "Content-Type": "application/json"}
        body: Dict[str, Any] = {"q": query, "num": num}
        if tbs:
            body["tbs"] = tbs
        with httpx.Client(timeout=30.0) as client:
            r = client.post(self.SERPER_URL, headers=headers, json=body)
        if r.status_code >= 400:
            return []
        organic = (r.json().get("organic") or [])
        return [{"title": it.get("title") or "", "link": it.get("link") or "", "snippet": it.get("snippet") or ""} for it in organic]


class Scraper:
    @staticmethod
    def fetch_text(url: str, snippet: str) -> str:
        if not url:
            return snippet or ""
        try:
            with httpx.Client(timeout=18.0, follow_redirects=True) as client:
                r = client.get(url, headers={"User-Agent":"Mozilla/5.0"})
            if r.status_code >= 400:
                return snippet or ""
            html = r.text
            txt = re.sub(r"<(script|style)[^>]*>.*?</\1>", " ", html, flags=re.S|re.I)
            txt = re.sub(r"<[^>]+>", " ", txt)
            txt = re.sub(r"\s+", " ", txt).strip()
            return txt[:16000] if txt else (snippet or "")
        except Exception:
            return snippet or ""


class OneClickAutomationEngine:
    """
    Description: Full one-click engine with soft-gated parser + full discovery/matching/ranking.
    Layer: L0-L9
    """

    def __init__(self) -> None:
        self._s = get_settings()
        self._store = SqliteStateStore()
        self._notifier = NotificationService(dry_run=not bool(self._s.twilio_account_sid))

        self._sanitize = SanitizeAgent()
        self._parser = ParserAgentService()
        self._parser_eval = ParserEvaluatorService()

        self._health = HealthService() if HealthService else None

    def _persist(self, st: OrchestrationState) -> None:
        d = _run_dir(st.run_id)
        _save_json(d / "state.json", st.model_dump())
        self._store.upsert_state(run_id=st.run_id, status=st.status, state=st.model_dump(), updated_at_utc=st.updated_at_utc)

    def load(self, run_id: str) -> Optional[Dict[str, Any]]:
        return self._store.get_state(run_id=run_id)

    def start_run(self, *, filename: str, data: bytes, prefs: Dict[str, Any]) -> OrchestrationState:
        st = OrchestrationState.new(env=self._s.environment, mode="agentic", git_sha=None)
        st.meta["preferences"] = prefs
        st.meta.setdefault("live_feed", [])
        st.meta.setdefault("job_scores", {})
        st.meta.setdefault("job_components", {})
        st.meta.setdefault("job_meta", {})
        LiveFeed.emit(st, layer="L1", agent="Dashboard", message="Run created. Starting autonomous pipeline…")
        self._persist(st)
        t = threading.Thread(target=self._run, args=(st.run_id, filename, data), daemon=True)
        t.start()
        return st

    def submit_action(self, *, run_id: str, action_type: str, payload: Dict[str, Any]) -> OrchestrationState:
        raw = self.load(run_id)
        if not raw:
            raise ValueError("run_id not found")
        st = OrchestrationState(**raw)
        st.meta["last_user_action"] = {"type": action_type, "payload": payload}
        LiveFeed.emit(st, layer="L5", agent="HITL", message=f"User action received: {action_type}")
        self._persist(st)
        t = threading.Thread(target=self._continue, args=(run_id,), daemon=True)
        t.start()
        return st

    def _run(self, run_id: str, filename: str, data: bytes) -> None:
        raw = self.load(run_id)
        if not raw:
            return
        st = OrchestrationState(**raw)
        run_dir = _run_dir(run_id)
        prefs = st.meta.get("preferences", {}) or {}

        # ---- L2 Extract ----
        st.start_step("l2_extract", layer_id="L2", tool_name="ResumeExtractor", input_ref={"filename": filename})
        LiveFeed.emit(st, layer="L2", agent="ParserAgent", message="Extracting resume text from upload…")
        try:
            resume_text = LocalResumeExtractor.extract_text(filename=filename, data=data)
        except Exception as e:
            st.end_step("l2_extract", status="failed", output_ref={}, message=str(e))
            st.status = "needs_human_approval"
            st.meta["pending_action"] = "resume_cleanup"
            LiveFeed.emit(st, layer="L2", agent="ParserAgent", message=f"Resume extraction failed: {e}")
            self._persist(st)
            return

        (run_dir / "resume_raw.txt").write_text(resume_text, encoding="utf-8")
        st.add_artifact("resume_raw", str(run_dir / "resume_raw.txt"), content_type="text/plain")
        st.end_step("l2_extract", status="ok", output_ref={"artifact_key":"resume_raw"}, message="extracted")
        self._persist(st)

        # ---- L0 Sanitize ----
        st.start_step("l0_sanitize", layer_id="L0", tool_name="SanitizeAgent", input_ref={})
        safe = self._sanitize.sanitize_before_llm(state=st, step_id="l0_sanitize", tool_name="sanitize_before_llm", user_text=resume_text, context="resume")
        if safe is None:
            st.status = "blocked"
            LiveFeed.emit(st, layer="L0", agent="SanitizeAgent", message="Prompt injection detected. Run blocked.")
            self._persist(st)
            return
        st.end_step("l0_sanitize", status="ok", output_ref={"sanitized": True}, message="pass")
        LiveFeed.emit(st, layer="L0", agent="SanitizeAgent", message="Security check passed.")
        self._persist(st)

        # ---- L2 Parse (single strong pass) ----
        st.start_step("l2_parse", layer_id="L2", tool_name="ParserAgentService", input_ref={"attempt": 1})
        extracted = self._parser.parse(raw_text=safe, orchestration_state=st, feedback=[])
        p = run_dir / "extracted_resume.json"
        _save_json(p, extracted.to_json_dict())
        st.add_artifact("extracted_resume", str(p), content_type="application/json")
        st.end_step("l2_parse", status="ok", output_ref={"artifact_key":"extracted_resume"}, message="parsed")

        # ---- L3 Evaluate (SOFT gate) ----
        # strict gate optional; default False for automation
        strict_gate = bool(prefs.get("resume_strict_gate", False))
        threshold = 0.80 if strict_gate else float(prefs.get("resume_threshold", 0.55))

        ev = self._parser_eval.evaluate(
            orchestration_state=st,
            raw_text=safe,
            extracted=extracted,
            target_id="resume_main",
            threshold=threshold,
            retry_count=0,
            max_retries=0,
        )

        score = float(getattr(ev, "evaluation_score", 0.0))
        LiveFeed.emit(st, layer="L3", agent="ParserEvaluator", message=f"Resume quality={score:.2f} (threshold={threshold:.2f}).")

        # If truly broken, stop; otherwise continue but allow optional cleanup
        if score < 0.30:
            st.status = "needs_human_approval"
            st.meta["pending_action"] = "resume_cleanup"
            LiveFeed.emit(st, layer="L3", agent="ParserEvaluator", message="Resume parsing too weak. Needs manual cleanup.")
            self._persist(st)
            return
        elif score < threshold:
            st.meta["pending_action"] = "resume_cleanup_optional"
            LiveFeed.emit(st, layer="L3", agent="ParserEvaluator", message="Proceeding automatically, but resume cleanup is recommended.")

        self._persist(st)

        # ---- L3 Discovery ----
        if not self._s.serper_api_key:
            st.status = "needs_human_approval"
            st.meta["pending_action"] = "missing_serper_key"
            LiveFeed.emit(st, layer="L3", agent="DiscoveryAgent", message="Missing SERPER_API_KEY in .env.")
            self._persist(st)
            return

        target_role = str(prefs.get("target_role","Data Scientist"))
        location = str(prefs.get("location","United States"))
        remote = bool(prefs.get("remote", True))
        wfo_ok = bool(prefs.get("wfo_ok", True))
        salary = str(prefs.get("salary","")).strip()
        visa_required = bool(prefs.get("visa_sponsorship_required", False))
        recency_hours = float(prefs.get("recency_hours", 36))
        max_jobs = int(prefs.get("max_jobs", 40))
        discovery_threshold = float(prefs.get("discovery_threshold", 0.70))
        max_refinements = int(prefs.get("max_refinements", 3))

        resume_skills = [s.lower() for s in (extracted.skills or [])]
        ats = _ats_score(safe)
        st.meta["ats_score"] = ats

        serper = SerperClient(self._s.serper_api_key)
        tbs = "qdr:d" if recency_hours <= 36 else None

        base_query = self._build_query(target_role, location, remote, wfo_ok, salary, visa_required, resume_skills)

        for attempt in range(max_refinements + 1):
            LiveFeed.emit(st, layer="L3", agent="DiscoveryAgent", message=f"Hunt attempt {attempt+1}: searching 8 boards…")
            results = self._discover_all(serper, base_query, tbs=tbs)

            # ---- L4 Scrape + score ----
            ranked = self._score_jobs(
                resume_text=safe,
                extracted=extracted,
                ats=ats,
                visa_required=visa_required,
                recency_hours=recency_hours,
                results=results[:max_jobs],
            )
            _save_json(run_dir / "ranking.json", ranked)
            st.add_artifact("ranking", str(run_dir / "ranking.json"), content_type="application/json")
            self._persist(st)

            if not ranked:
                conf = 0.0
            else:
                top = float(ranked[0]["interview_chance_score"])
                avg = sum(float(x["interview_chance_score"]) for x in ranked[:20]) / max(1, min(20, len(ranked)))
                conf = min(1.0, 0.65*top + 0.35*avg)

            LiveFeed.emit(st, layer="L5", agent="EvaluatorAgent", message=f"Discovery confidence={conf:.2f} (need ≥ {discovery_threshold:.2f}).")

            if ranked and float(ranked[0]["interview_chance_score"]) >= discovery_threshold:
                st.status = "needs_human_approval"
                st.meta["pending_action"] = "review_ranking"
                LiveFeed.emit(st, layer="L1", agent="Dashboard", message="Ranking ready for review (HITL).")
                self._persist(st)
                return

            if attempt >= max_refinements:
                st.status = "needs_human_approval"
                st.meta["pending_action"] = "low_confidence_discovery"
                LiveFeed.emit(st, layer="L5", agent="EvaluatorAgent", message="Low confidence after retries. Needs guidance.")
                self._persist(st)
                return

            base_query = self._refine_query(base_query, ranked, resume_skills, visa_required, target_role, location)
            LiveFeed.emit(st, layer="L3", agent="DiscoveryAgent", message=f"Refining query → {base_query[:160]}")
            self._persist(st)

    def _continue(self, run_id: str) -> None:
        raw = self.load(run_id)
        if not raw:
            return
        st = OrchestrationState(**raw)
        run_dir = _run_dir(run_id)
        pending = st.meta.get("pending_action")
        action = (st.meta.get("last_user_action") or {}).get("type")
        payload = (st.meta.get("last_user_action") or {}).get("payload") or {}

        # NEW: Resume cleanup submit
        if pending in ("resume_cleanup", "resume_cleanup_optional") and action == "resume_cleanup_submit":
            new_text = str(payload.get("resume_text","")).strip()
            if not new_text:
                LiveFeed.emit(st, layer="L5", agent="HITL", message="Resume cleanup submitted but text was empty.")
                self._persist(st)
                return

            (run_dir / "resume_raw.txt").write_text(new_text, encoding="utf-8")
            st.add_artifact("resume_raw", str(run_dir / "resume_raw.txt"), content_type="text/plain")
            st.meta["pending_action"] = None
            st.status = "running"
            LiveFeed.emit(st, layer="L2", agent="ParserAgent", message="Resume updated by user. Restarting pipeline…")
            self._persist(st)

            # restart the run using updated text as txt bytes (no need for original upload)
            self._run(run_id, "resume.txt", new_text.encode("utf-8"))
            return

        # Ranking approval/reject can be wired next (your next step).
        LiveFeed.emit(st, layer="L5", agent="HITL", message=f"No handler for pending={pending}, action={action}")
        self._persist(st)

    def _build_query(self, target_role: str, location: str, remote: bool, wfo_ok: bool, salary: str, visa_required: bool, skills: List[str]) -> str:
        intent = []
        if remote: intent.append("remote")
        if wfo_ok: intent.append("hybrid")
        if not intent: intent.append("on-site")
        skill_str = " ".join(skills[:6])
        salary_part = f'"{salary}"' if salary else ""
        visa_part = '"visa sponsorship" OR h1b OR opt OR cpt' if visa_required else ""
        return f'{target_role} {location} {" ".join(intent)} {salary_part} {skill_str} ({visa_part}) apply'

    def _discover_all(self, serper: SerperClient, base_query: str, tbs: Optional[str]) -> List[Dict[str, Any]]:
        seen = set()
        out: List[Dict[str, Any]] = []
        for name, domain in JOB_BOARDS:
            q = f"{base_query} site:{domain}"
            items = serper.search(q, num=10, tbs=tbs)
            for it in items:
                link = it.get("link") or ""
                if not link or link in seen:
                    continue
                seen.add(link)
                it["board"] = name
                out.append(it)
        return out

    def _score_jobs(self, *, resume_text: str, extracted: ExtractedResume, ats: float, visa_required: bool, recency_hours: float, results: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        ranked: List[Dict[str, Any]] = []
        res_tokens = {}
        for t in _tokenize(resume_text):
            res_tokens[t] = res_tokens.get(t, 0) + 1
        exp_text = " ".join([" ".join(x.bullets) for x in (extracted.experience or [])]) if extracted.experience else resume_text
        exp_tokens = {}
        for t in _tokenize(exp_text):
            exp_tokens[t] = exp_tokens.get(t, 0) + 1
        resume_skills = [s.lower() for s in (extracted.skills or [])]

        for it in results:
            snippet = it.get("snippet") or ""
            rh = _parse_recency_hours(snippet)
            if rh is not None and rh > recency_hours:
                continue

            url = it.get("link") or ""
            board = it.get("board") or "unknown"
            title = it.get("title") or ""

            job_text = Scraper.fetch_text(url, snippet)
            low = job_text.lower()

            # visa filter
            v_ok = not any(x in low for x in VISA_NEGATIVE)
            if visa_required and not v_ok:
                continue

            # skills overlap (from resume skills present in job text)
            present = [s for s in resume_skills if s and s in low]
            overlap = len(set(present)) / max(1, len(set(resume_skills))) if resume_skills else 0.0

            # exp alignment cosine
            job_tokens = {}
            for t in _tokenize(job_text):
                job_tokens[t] = job_tokens.get(t, 0) + 1
            exp_align = _cosine(exp_tokens, job_tokens)

            market = 1.0
            if "applicants" in snippet.lower():
                m = re.search(r"(\d+)\+?\s*applicants", snippet.lower())
                if m:
                    n = int(m.group(1))
                    market = 1.0 + min(1.5, n/200.0)

            score = _compute_interview_chance(overlap, exp_align, ats, market)
            if visa_required and any(x in low for x in VISA_POSITIVE):
                score = min(1.0, score + 0.05)

            ranked.append({
                "title": title,
                "board": board,
                "url": url,
                "recency_hours": rh,
                "visa_ok": v_ok,
                "skill_overlap": overlap,
                "experience_alignment": exp_align,
                "ats_score": ats,
                "market_factor": market,
                "interview_chance_score": score,
                "overall_match_percent": round(score*100.0, 2),
                "matched_skills": present[:12],
                "rationale": [
                    f"SkillOverlap={overlap:.2f}",
                    f"ExperienceAlignment={exp_align:.2f}",
                    f"ATS={ats:.2f}",
                    f"MarketFactor={market:.2f}",
                    ("VisaOK" if v_ok else "NoSponsorship"),
                ]
            })

        ranked.sort(key=lambda x: float(x["interview_chance_score"]), reverse=True)
        for i, r in enumerate(ranked, start=1):
            r["rank"] = i
        return ranked

    def _refine_query(self, base_query: str, ranked: List[Dict[str, Any]], resume_skills: List[str], visa_required: bool, target_role: str, location: str) -> str:
        top = ranked[0] if ranked else {}
        top_skills = (top.get("matched_skills") or [])[:6]
        hint = " ".join(list(dict.fromkeys([*top_skills, *resume_skills]))[:8])
        visa = '("visa sponsorship" OR h1b OR opt)' if visa_required else ""
        return f"{target_role} {location} {hint} {visa} apply"


ENGINE = OneClickAutomationEngine()
'''
backup_write("src/careeragent/orchestration/engine.py", ENGINE_PATCH)

backup_write("src/careeragent/orchestration/__init__.py", "from .state import OrchestrationState\nfrom .engine import ENGINE, OneClickAutomationEngine\n")

# ---------------------------
# 4) Update dashboard: add Resume Cleanup HITL UI + L0-L9 layer panels
# ---------------------------
DASHBOARD_PATCH = r'''
from __future__ import annotations

import json
import os
import sys
from pathlib import Path
from typing import Any, Dict, Optional

import requests
import streamlit as st

REPO_ROOT = Path(__file__).resolve().parents[2]
SRC = REPO_ROOT / "src"
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

DEFAULT_API = os.getenv("API_URL", "http://127.0.0.1:8000")


def _safe_json(resp: requests.Response) -> Dict[str, Any]:
    try:
        return resp.json()
    except Exception:
        return {"_raw": resp.text[:1500], "_status_code": resp.status_code}


def _api_get(api: str, path: str, timeout: int = 25) -> requests.Response:
    return requests.get(f"{api}{path}", timeout=timeout)


def _api_post(api: str, path: str, timeout: int = 30, **kwargs) -> requests.Response:
    return requests.post(f"{api}{path}", timeout=timeout, **kwargs)


def _exists(path: Optional[str]) -> bool:
    return bool(path) and Path(path).exists()


def main() -> None:
    st.set_page_config(page_title="CareerAgent-AI Mission Control", layout="wide")
    st.title("CareerAgent-AI — Mission Control (One-Click Automation)")
    st.caption("Upload resume → autonomous ingestion + discovery + ranking → HITL approvals → drafts + dossier downloads")

    api = st.sidebar.text_input("API Base URL", value=DEFAULT_API)

    # Backend status
    st.sidebar.divider()
    st.sidebar.subheader("Backend")
    try:
        h = _api_get(api, "/health", timeout=3)
        if h.status_code == 200:
            st.sidebar.success("🟢 API Online")
        else:
            st.sidebar.warning(f"🟠 API issue ({h.status_code})")
    except Exception as e:
        st.sidebar.error("🔴 API Offline")
        st.sidebar.caption(str(e))
        st.stop()

    # Inputs
    st.sidebar.divider()
    st.sidebar.subheader("Resume Upload")
    resume_file = st.sidebar.file_uploader("Upload Resume (PDF/TXT/DOCX)", type=["pdf", "txt", "docx"])

    st.sidebar.subheader("Preferences")
    target_role = st.sidebar.text_input("Target role", value="Data Scientist")
    country = st.sidebar.text_input("Country", value="US")
    location = st.sidebar.text_input("Location", value="United States")
    remote = st.sidebar.checkbox("Remote preferred", value=True)
    wfo_ok = st.sidebar.checkbox("On-site/WFO acceptable", value=True)
    salary = st.sidebar.text_input("Salary target (optional)", value="")
    visa_required = st.sidebar.checkbox("Visa sponsorship required (F1/OPT)", value=False)

    recency_hours = st.sidebar.slider("Only jobs posted within (hours)", 12, 168, 36, 6)
    max_jobs = st.sidebar.slider("Max jobs to score per run", 10, 80, 40, 5)

    st.sidebar.subheader("Autonomy Controls")
    discovery_threshold = st.sidebar.slider("Discovery confidence threshold", 0.50, 0.90, 0.70, 0.05)
    max_refinements = st.sidebar.slider("Max query refinements", 1, 6, 3, 1)

    # NEW: soft gate controls
    resume_strict_gate = st.sidebar.checkbox("Strict resume gate (stop on low parse)", value=False)
    resume_threshold = st.sidebar.slider("Resume quality threshold (soft gate)", 0.35, 0.90, 0.55, 0.05)

    run_btn = st.sidebar.button("🚀 RUN ONE-CLICK", type="primary", use_container_width=True, disabled=(resume_file is None))

    st.sidebar.divider()
    st.sidebar.subheader("Existing Run")
    run_id_in = st.sidebar.text_input("Run ID", value=st.session_state.get("run_id", ""))

    if run_btn:
        prefs = {
            "target_role": target_role.strip() or "Data Scientist",
            "country": country.strip() or "US",
            "location": location.strip() or "United States",
            "remote": bool(remote),
            "wfo_ok": bool(wfo_ok),
            "salary": salary.strip(),
            "visa_sponsorship_required": bool(visa_required),
            "recency_hours": float(recency_hours),
            "max_jobs": int(max_jobs),
            "discovery_threshold": float(discovery_threshold),
            "max_refinements": int(max_refinements),
            "resume_strict_gate": bool(resume_strict_gate),
            "resume_threshold": float(resume_threshold),
        }
        files = {"resume": (resume_file.name, resume_file.getvalue())}
        data = {"preferences_json": json.dumps(prefs)}
        r = _api_post(api, "/analyze", files=files, data=data, timeout=180)
        if r.status_code >= 400:
            st.error(f"/analyze failed: {r.status_code}\n\n{r.text[:1500]}")
            st.stop()
        out = _safe_json(r)
        st.session_state["run_id"] = out["run_id"]
        st.success(f"Run started: {out['run_id']} (status: {out.get('status')})")

    run_id = st.session_state.get("run_id") or run_id_in.strip()
    if not run_id:
        st.info("Upload resume and click RUN, or paste a run_id.")
        return

    # Poll
    _ = st.button("🔄 Refresh", use_container_width=True)
    r = _api_get(api, f"/status/{run_id}", timeout=25)
    if r.status_code != 200:
        st.warning(f"Run not found yet ({r.status_code}).")
        return
    state = _safe_json(r)

    status = state.get("status", "unknown")
    meta = state.get("meta", {}) or {}
    pending = meta.get("pending_action")
    feed = meta.get("live_feed", []) or []
    steps = state.get("steps", []) or []
    artifacts = state.get("artifacts", {}) or {}
    evals = state.get("evaluations", []) or []

    c1, c2, c3 = st.columns([1,1,1])
    with c1: st.metric("Run ID", run_id)
    with c2: st.metric("Status", status)
    with c3: st.metric("Pending", str(pending))

    # Layer map (aligned with your architecture)
    with st.expander("Architecture Layer Map (L0–L9)", expanded=False):
        st.markdown("""
- **L0** Security & Guardrails  
- **L1** Mission Control (UI)  
- **L2** Parsing & Profile Extraction  
- **L3** Discovery / Connectors  
- **L4** Scrape + Matching / Vectorization  
- **L5** Evaluator + Ranking + HITL gates  
- **L6** Drafting (Resume/Cover/Answers)  
- **L7** Execution (Apply/Notifications)  
- **L8** Tracking (Status/CRM/DB)  
- **L9** Analytics + XAI + Dossier Export
        """)

    # Feed
    st.markdown("### Live Agent Feed")
    for ev in feed[-180:]:
        st.write(f"**[{ev.get('layer')} {ev.get('agent')}]** {ev.get('message')}")

    # Evaluations
    with st.expander("Evaluations (Scores + Reasons)", expanded=False):
        if not evals:
            st.caption("No evaluations yet.")
        else:
            for e in evals[-10:]:
                st.write(f"**{e.get('layer_id')} / {e.get('target_id')}** score={e.get('evaluation_score'):.2f} threshold={e.get('threshold')}")
                fb = e.get("feedback") or []
                if fb:
                    st.write("- " + "\n- ".join(fb[:6]))

    # Resume cleanup HITL
    if status == "needs_human_approval" and pending in ("resume_cleanup", "resume_cleanup_optional"):
        st.markdown("## Human-in-the-Loop: Resume Cleanup")
        resume_ref = (artifacts.get("resume_raw") or {}).get("path")
        current_text = ""
        if resume_ref and Path(resume_ref).exists():
            current_text = Path(resume_ref).read_text(encoding="utf-8", errors="ignore")

        st.info("Your resume is parseable, but quality is below target. You can paste an improved version (add email/phone + clearer headings) and continue automatically.")
        new_text = st.text_area("Paste improved resume text (ATS headings + contact)", value=current_text, height=220)

        if st.button("✅ Submit cleaned resume & continue", type="primary"):
            rr = _api_post(api, f"/action/{run_id}", json={"action_type": "resume_cleanup_submit", "payload": {"resume_text": new_text}}, timeout=60)
            if rr.status_code >= 400:
                st.error(rr.text[:1200])
            else:
                st.success("Submitted. Refresh in a few seconds.")

    # Ranking
    st.markdown("### Ranking")
    ranking_ref = artifacts.get("ranking")
    if ranking_ref and _exists(ranking_ref.get("path")):
        ranking = json.loads(Path(ranking_ref["path"]).read_text(encoding="utf-8"))
        st.dataframe(
            [{
                "rank": x.get("rank"),
                "score_%": x.get("overall_match_percent"),
                "title": x.get("title"),
                "board": x.get("board"),
                "recency_h": x.get("recency_hours"),
                "visa_ok": x.get("visa_ok"),
                "url": x.get("url"),
            } for x in ranking],
            use_container_width=True
        )
    else:
        st.caption("Ranking not available yet.")

    with st.expander("Full State JSON", expanded=False):
        st.json(state)


if __name__ == "__main__":
    main()
'''
backup_write("app/ui/dashboard.py", DASHBOARD_PATCH)

# ---------------------------
# 5) app/main.py robust bootstrap (keeps app import stable)
# ---------------------------
APP_MAIN = r'''
from __future__ import annotations

import sys
from pathlib import Path

ROOT = Path(__file__).resolve().parents[1]
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

SRC = ROOT / "src"
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

from app.ui.dashboard import main
main()
'''
backup_write("app/main.py", APP_MAIN)

print("\n✅ PATCH COMPLETE.")
print("Restart backend + frontend now.\n")
print("Backend:")
print("  PYTHONPATH=src uv run python -m uvicorn careeragent.api.main:app --host 127.0.0.1 --port 8000 --reload")
print("Frontend:")
print("  API_URL=http://127.0.0.1:8000 PYTHONPATH='.:src' uv run streamlit run app/main.py --server.port 8501")

✅ CWD = /Users/ganeshprasadbhandari/Documents/D_drive/clark/careeragent-ai
BACKUP: /Users/ganeshprasadbhandari/Documents/D_drive/clark/careeragent-ai/src/careeragent/agents/parser_agent_service.py.bak_20260221_011632
WROTE: /Users/ganeshprasadbhandari/Documents/D_drive/clark/careeragent-ai/src/careeragent/agents/parser_agent_service.py
BACKUP: /Users/ganeshprasadbhandari/Documents/D_drive/clark/careeragent-ai/src/careeragent/agents/parser_evaluator_service.py.bak_20260221_011632
WROTE: /Users/ganeshprasadbhandari/Documents/D_drive/clark/careeragent-ai/src/careeragent/agents/parser_evaluator_service.py
BACKUP: /Users/ganeshprasadbhandari/Documents/D_drive/clark/careeragent-ai/src/careeragent/orchestration/engine.py.bak_20260221_011632
WROTE: /Users/ganeshprasadbhandari/Documents/D_drive/clark/careeragent-ai/src/careeragent/orchestration/engine.py
BACKUP: /Users/ganeshprasadbhandari/Documents/D_drive/clark/careeragent-ai/src/careeragent/orchestration/__init__.py.bak_20260221_011632
WROTE

In [6]:
%pwd

'/Users/ganeshprasadbhandari/Documents/D_drive/clark/careeragent-ai'

In [7]:
from __future__ import annotations

import os
import json
from pathlib import Path
from datetime import datetime

# Auto-chdir to repo root even when notebook runs in notebooks_v2
ROOT = Path.cwd().resolve()
if ROOT.name == "notebooks_v2":
    os.chdir(ROOT.parent)
ROOT = Path.cwd().resolve()
assert (ROOT / "src").exists(), f"Not at repo root. CWD={ROOT}"
print("✅ CWD =", ROOT)

def backup_write(rel_path: str, content: str) -> None:
    p = ROOT / rel_path
    p.parent.mkdir(parents=True, exist_ok=True)
    if p.exists():
        ts = datetime.now().strftime("%Y%m%d_%H%M%S")
        bak = p.with_suffix(p.suffix + f".bak_{ts}")
        bak.write_text(p.read_text(encoding="utf-8"), encoding="utf-8")
        print("BACKUP:", bak)
    p.write_text(content, encoding="utf-8")
    print("WROTE:", p)

# Ensure packages
(ROOT / "src/careeragent/services").mkdir(parents=True, exist_ok=True)
(ROOT / "src/careeragent/agents").mkdir(parents=True, exist_ok=True)
(ROOT / "src/careeragent/orchestration").mkdir(parents=True, exist_ok=True)
(ROOT / "app/ui").mkdir(parents=True, exist_ok=True)
(ROOT / "app").mkdir(parents=True, exist_ok=True)

(ROOT / "src/careeragent/services/__init__.py").write_text("", encoding="utf-8")
(ROOT / "src/careeragent/agents/__init__.py").write_text("", encoding="utf-8")
(ROOT / "src/careeragent/orchestration/__init__.py").write_text("", encoding="utf-8")
(ROOT / "app/__init__.py").write_text("", encoding="utf-8")
(ROOT / "app/ui/__init__.py").write_text("", encoding="utf-8")

# -------------------------
# services/db_service.py
# -------------------------
backup_write("src/careeragent/services/db_service.py", r'''
from __future__ import annotations

import json
import sqlite3
from pathlib import Path
from typing import Any, Dict, Optional

from careeragent.config import artifacts_root


class SqliteStateStore:
    """
    Description: Local-first persistence for OrchestrationState.
    Layer: L8
    Input: run state snapshots
    Output: sqlite persisted state for polling
    """

    def __init__(self) -> None:
        db_dir = artifacts_root() / "db"
        db_dir.mkdir(parents=True, exist_ok=True)
        self._db_path = db_dir / "careeragent.db"
        self._init_schema()

    def _init_schema(self) -> None:
        with sqlite3.connect(self._db_path) as con:
            con.execute("""
                CREATE TABLE IF NOT EXISTS runs (
                    run_id TEXT PRIMARY KEY,
                    status TEXT NOT NULL,
                    state_json TEXT NOT NULL,
                    updated_at_utc TEXT NOT NULL
                )
            """)
            con.execute("""
                CREATE TABLE IF NOT EXISTS actions (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    run_id TEXT NOT NULL,
                    action_type TEXT NOT NULL,
                    payload_json TEXT NOT NULL,
                    created_at_utc TEXT NOT NULL
                )
            """)
            con.commit()

    def upsert_state(self, *, run_id: str, status: str, state: Dict[str, Any], updated_at_utc: str) -> None:
        with sqlite3.connect(self._db_path) as con:
            con.execute("""
                INSERT INTO runs(run_id, status, state_json, updated_at_utc)
                VALUES(?,?,?,?)
                ON CONFLICT(run_id) DO UPDATE SET
                    status=excluded.status,
                    state_json=excluded.state_json,
                    updated_at_utc=excluded.updated_at_utc
            """, (run_id, status, json.dumps(state), updated_at_utc))
            con.commit()

    def get_state(self, *, run_id: str) -> Optional[Dict[str, Any]]:
        with sqlite3.connect(self._db_path) as con:
            cur = con.execute("SELECT state_json FROM runs WHERE run_id=?", (run_id,))
            row = cur.fetchone()
            return json.loads(row[0]) if row else None

    def insert_action(self, *, run_id: str, action_type: str, payload: Dict[str, Any], created_at_utc: str) -> None:
        with sqlite3.connect(self._db_path) as con:
            con.execute(
                "INSERT INTO actions(run_id, action_type, payload_json, created_at_utc) VALUES(?,?,?,?)",
                (run_id, action_type, json.dumps(payload), created_at_utc),
            )
            con.commit()
''')

# -------------------------
# services/health_service.py (quota + tracing hooks)
# -------------------------
backup_write("src/careeragent/services/health_service.py", r'''
from __future__ import annotations

from dataclasses import dataclass
from typing import Optional

from careeragent.orchestration.state import OrchestrationState


@dataclass
class QuotaManager:
    """
    Description: Tracks quota errors for external APIs and blocks runs.
    Layer: L0
    """
    serper_quota_exceeded: bool = False
    last_error: Optional[str] = None

    def handle_serper_response(self, *, state: OrchestrationState, step_id: str, status_code: int, tool_name: str, error_detail: str) -> bool:
        if status_code == 403:
            self.serper_quota_exceeded = True
            self.last_error = error_detail
            state.status = "blocked"
            state.meta["run_failure_code"] = "API_FAILURE"
            state.meta["run_failure_provider"] = "serper"
            state.meta["run_failure_detail"] = error_detail
            # mark step blocked if possible
            try:
                state.end_step(step_id, status="blocked", output_ref={"error": "quota_exceeded"}, message="serper_quota_exceeded")
            except Exception:
                pass
            return True
        return False


class HealthService:
    """
    Description: Local health + tracing wiring.
    Layer: L0
    """
    def __init__(self) -> None:
        self.quota = QuotaManager()

    def load_env(self, *, dotenv_path: str) -> None:
        # No-op: settings already load .env via pydantic-settings in your repo.
        return

    def enable_langsmith_tracing(self, *, project: str) -> None:
        # No-op: optional integration later.
        return
''')

# -------------------------
# services/learning_resource_service.py
# -------------------------
backup_write("src/careeragent/services/learning_resource_service.py", r'''
from __future__ import annotations

from typing import Any, Dict, List, Optional
import httpx


class LearningResourceService:
    """
    Description: Fetch tutorials / YouTube / docs for missing skills using Serper.
    Layer: L9
    Input: skill list
    Output: list of learning links
    """

    SERPER_URL = "https://google.serper.dev/search"

    def __init__(self, *, serper_api_key: Optional[str]) -> None:
        self._key = (serper_api_key or "").strip()

    def available(self) -> bool:
        return bool(self._key)

    def search_links(self, *, query: str, num: int = 5) -> List[Dict[str, Any]]:
        if not self.available():
            return []
        headers = {"X-API-KEY": self._key, "Content-Type": "application/json"}
        with httpx.Client(timeout=25.0) as client:
            r = client.post(self.SERPER_URL, headers=headers, json={"q": query, "num": num})
        if r.status_code >= 400:
            return []
        organic = (r.json().get("organic") or [])
        out = []
        for it in organic[:num]:
            out.append({"title": it.get("title"), "link": it.get("link"), "snippet": it.get("snippet")})
        return out

    def build_learning_plan(self, *, missing_skills: List[str]) -> Dict[str, Any]:
        plan: Dict[str, Any] = {}
        for s in missing_skills[:12]:
            skill = str(s).strip()
            if not skill:
                continue
            plan[skill] = {
                "youtube": self.search_links(query=f"{skill} tutorial youtube", num=3),
                "docs": self.search_links(query=f"{skill} official documentation", num=3),
                "course": self.search_links(query=f"best course to learn {skill}", num=3),
            }
        return plan
''')

# -------------------------
# agents/parser_agent_service.py (deterministic intake bundle)
# -------------------------
backup_write("src/careeragent/agents/parser_agent_service.py", r'''
from __future__ import annotations

import re
from typing import Any, List, Optional
from pydantic import BaseModel, Field, ConfigDict


class ContactInfo(BaseModel):
    model_config = ConfigDict(extra="ignore")
    email: Optional[str] = None
    phone: Optional[str] = None
    linkedin: Optional[str] = None
    github: Optional[str] = None


class ExperienceItem(BaseModel):
    model_config = ConfigDict(extra="ignore")
    bullets: List[str] = Field(default_factory=list)


class ExtractedResume(BaseModel):
    """
    Description: L2 Intake Bundle (profile extracted from raw resume).
    Layer: L2
    Input: raw resume text
    Output: structured profile JSON
    """
    model_config = ConfigDict(extra="ignore")
    name: Optional[str] = None
    contact: ContactInfo = Field(default_factory=ContactInfo)
    summary: Optional[str] = None
    skills: List[str] = Field(default_factory=list)
    experience: List[ExperienceItem] = Field(default_factory=list)
    raw_text: Optional[str] = None
    warnings: List[str] = Field(default_factory=list)

    def to_json_dict(self) -> dict[str, Any]:
        return self.model_dump()


COMMON_SKILLS = [
    "python","sql","fastapi","docker","kubernetes","mlflow","dvc","aws","azure","gcp",
    "pytorch","tensorflow","scikit-learn","pandas","numpy","spark","databricks","snowflake",
    "langchain","langgraph","rag","faiss","chroma","terraform","github actions","kafka","airflow"
]


def _find_email(text: str) -> Optional[str]:
    m = re.search(r"[\w\.-]+@[\w\.-]+\.\w+", text or "")
    return m.group(0) if m else None


def _find_phone(text: str) -> Optional[str]:
    m = re.search(r"(\+?\d{1,3}[\s\-]?)?(\(?\d{3}\)?[\s\-]?)\d{3}[\s\-]?\d{4}", text or "")
    return m.group(0).strip() if m else None


def _find_url(text: str, kw: str) -> Optional[str]:
    m = re.search(rf"(https?://[^\s]*{kw}[^\s]*)", text or "", flags=re.I)
    return m.group(1) if m else None


def _extract_name(text: str) -> Optional[str]:
    lines = [ln.strip() for ln in (text or "").splitlines() if ln.strip()]
    if not lines:
        return None
    cand = lines[0]
    if "@" in cand:
        return None
    return cand[:60]


def _extract_skills(text: str) -> List[str]:
    low = (text or "").lower()
    found = []
    for s in COMMON_SKILLS:
        if s in low:
            found.append(s)
    # also parse a "skills:" line
    m = re.search(r"skills\s*[:\-]\s*(.+)", text or "", flags=re.I)
    if m:
        parts = re.split(r"[,\|/•\n]+", m.group(1))
        found.extend([p.strip().lower() for p in parts if p.strip()])
    # unique + clean
    out = []
    for x in found:
        x = re.sub(r"\s+", " ", x.strip())
        if 2 <= len(x) <= 40 and x not in out:
            out.append(x)
    return out[:60]


class ParserAgentService:
    """
    Description: Deterministic ParserAgent (CareerOS-style). Never blocks automation.
    Layer: L2
    Input: raw resume text
    Output: ExtractedResume
    """
    def parse(self, *, raw_text: str, orchestration_state: Any, feedback: List[str] | None = None) -> ExtractedResume:
        text = raw_text or ""
        email = _find_email(text)
        phone = _find_phone(text)
        linkedin = _find_url(text, "linkedin")
        github = _find_url(text, "github")

        skills = _extract_skills(text)

        # summary: first 2-3 non-empty lines after name
        lines = [ln.strip() for ln in text.splitlines() if ln.strip()]
        name = _extract_name(text)
        summary = None
        if len(lines) >= 3:
            summary = " ".join(lines[1:3])[:400]

        # experience bullets: take lines starting with dash/bullet
        bullets = []
        for ln in lines:
            if ln.startswith(("-", "•")) and len(ln) > 15:
                bullets.append(ln.strip("-• ").strip())
        exp = [ExperienceItem(bullets=bullets[:12])] if bullets else []

        warnings = []
        if not (email or phone):
            warnings.append("Contact info missing (email/phone).")
        if len(skills) < 6:
            warnings.append("Low extracted skills; add a clearer Skills section.")
        if not bullets:
            warnings.append("No bullet points detected; add bullets under Experience for ATS.")

        return ExtractedResume(
            name=name,
            contact=ContactInfo(email=email, phone=phone, linkedin=linkedin, github=github),
            summary=summary,
            skills=skills,
            experience=exp,
            raw_text=text,
            warnings=warnings,
        )
''')

# -------------------------
# agents/parser_evaluator_service.py (SOFT gate)
# -------------------------
backup_write("src/careeragent/agents/parser_evaluator_service.py", r'''
from __future__ import annotations

import re
from typing import Any, List
from careeragent.agents.parser_agent_service import ExtractedResume


def _ats_structure_score(text: str) -> float:
    t = (text or "").lower()
    score = 0.0
    for h in ["summary","skills","experience","education","projects"]:
        if h in t:
            score += 0.14
    if re.search(r"[\w\.-]+@[\w\.-]+\.\w+", t):
        score += 0.15
    if "-" in text or "•" in text:
        score += 0.15
    if len(text) > 1200:
        score += 0.15
    return max(0.0, min(1.0, score))


class ParserEvaluatorService:
    """
    Description: Evaluates extracted profile quality (SOFT gate).
    Layer: L3
    """
    def evaluate(
        self,
        *,
        orchestration_state: Any,
        raw_text: str,
        extracted: ExtractedResume,
        target_id: str,
        threshold: float = 0.55,
        retry_count: int = 0,
        max_retries: int = 0,
    ) -> Any:
        fb: List[str] = []
        contact_ok = bool(extracted.contact.email or extracted.contact.phone)
        skills_ok = len(extracted.skills or []) >= 6
        exp_ok = bool(extracted.experience and extracted.experience[0].bullets)
        ats = _ats_structure_score(raw_text)

        score = (0.15*(1.0 if contact_ok else 0.0) +
                 0.40*min(1.0, len(extracted.skills or [])/12.0) +
                 0.30*(1.0 if exp_ok else 0.0) +
                 0.15*ats)

        if not contact_ok:
            fb.append("Contact missing. Continue anyway; but adding email/phone improves ATS + recruiter trust.")
        if not exp_ok:
            fb.append("No bullet points detected. Continue anyway; but bullets improve ATS scanability.")
        if not skills_ok:
            fb.append("Skills list looks thin. Continue anyway; add more relevant tools/keywords.")
        if ats < 0.5:
            fb.append("ATS structure weak. Add headings and bullets.")

        return orchestration_state.record_evaluation(
            layer_id="L3",
            target_id=target_id,
            generator_agent="parser_agent_service",
            evaluator_agent="parser_evaluator_service",
            evaluation_score=float(score),
            threshold=float(threshold),
            feedback=fb,
            retry_count=int(retry_count),
            max_retries=int(max_retries),
            interview_chance={
                "weights": {"w1_skill_overlap": 0.45, "w2_experience_alignment": 0.35, "w3_ats_score": 0.20},
                "components": {"skill_overlap": 0.0, "experience_alignment": 0.0, "ats_score": float(ats), "market_competition_factor": 1.0},
                "interview_chance_score": float(0.20*ats),
            },
        )
''')

# -------------------------
# orchestration/engine.py (FULL automation: L2 intake → L3 hunt 8 boards → L4 scrape+match → L5 rank gate → L6 drafts → learning plan)
# -------------------------
backup_write("src/careeragent/orchestration/engine.py", r'''
from __future__ import annotations

import json
import re
import threading
from dataclasses import dataclass
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple
from uuid import uuid4

import httpx

from careeragent.config import artifacts_root, get_settings
from careeragent.orchestration.state import OrchestrationState
from careeragent.services.db_service import SqliteStateStore
from careeragent.services.health_service import HealthService
from careeragent.services.notification_service import NotificationService
from careeragent.services.learning_resource_service import LearningResourceService

from careeragent.agents.security_agent import SanitizeAgent
from careeragent.agents.parser_agent_service import ParserAgentService, ExtractedResume
from careeragent.agents.parser_evaluator_service import ParserEvaluatorService


@dataclass(frozen=True)
class JobBoard:
    name: str
    domain: str


DEFAULT_JOB_BOARDS: Tuple[JobBoard, ...] = (
    JobBoard("LinkedIn Jobs", "linkedin.com/jobs"),
    JobBoard("Indeed", "indeed.com"),
    JobBoard("Glassdoor", "glassdoor.com"),
    JobBoard("ZipRecruiter", "ziprecruiter.com"),
    JobBoard("Monster", "monster.com"),
    JobBoard("Dice", "dice.com"),
    JobBoard("Lever", "jobs.lever.co"),
    JobBoard("Greenhouse", "boards.greenhouse.io"),
)

VISA_NEGATIVE = ("unable to sponsor","cannot sponsor","no sponsorship","do not sponsor","not sponsor","without sponsorship","no visa","cannot provide visa")
VISA_POSITIVE = ("visa sponsorship","h1b","opt","cpt","stem opt","work visa")

COMMON_SKILL_DICTIONARY = [
    "python","sql","fastapi","docker","kubernetes","mlflow","dvc","aws","azure","gcp","pytorch",
    "tensorflow","scikit-learn","pandas","numpy","spark","databricks","snowflake","airflow","kafka",
    "langchain","langgraph","rag","faiss","chroma","terraform","github actions","postgres","redis"
]


def _run_dir(run_id: str) -> Path:
    d = artifacts_root() / "runs" / run_id
    d.mkdir(parents=True, exist_ok=True)
    return d


def _save_json(p: Path, obj: Any) -> None:
    p.parent.mkdir(parents=True, exist_ok=True)
    p.write_text(json.dumps(obj, indent=2), encoding="utf-8")


class LiveFeed:
    @staticmethod
    def emit(st: OrchestrationState, *, layer: str, agent: str, message: str) -> None:
        st.meta.setdefault("live_feed", [])
        st.meta["live_feed"].append({"layer": layer, "agent": agent, "message": message})
        st.touch()


class LocalResumeExtractor:
    @staticmethod
    def extract_text(*, filename: str, data: bytes) -> str:
        name = (filename or "").lower()
        if name.endswith(".txt"):
            return data.decode("utf-8", errors="replace")
        if name.endswith(".pdf"):
            from pypdf import PdfReader  # type: ignore
            import io
            reader = PdfReader(io.BytesIO(data))
            return "\n".join([(pg.extract_text() or "") for pg in reader.pages])
        if name.endswith(".docx"):
            import docx  # type: ignore
            import io
            d = docx.Document(io.BytesIO(data))
            return "\n".join([p.text for p in d.paragraphs if p.text])
        return data.decode("utf-8", errors="replace")


class SerperDiscovery:
    SERPER_URL = "https://google.serper.dev/search"

    def __init__(self, *, api_key: str, health: HealthService) -> None:
        self._key = api_key
        self._health = health

    def search(self, *, st: OrchestrationState, step_id: str, query: str, num: int = 10, tbs: Optional[str] = None) -> List[Dict[str, Any]]:
        headers = {"X-API-KEY": self._key, "Content-Type": "application/json"}
        body: Dict[str, Any] = {"q": query, "num": num}
        if tbs:
            body["tbs"] = tbs
        with httpx.Client(timeout=30.0) as client:
            r = client.post(self.SERPER_URL, headers=headers, json=body)

        if self._health.quota.handle_serper_response(
            state=st, step_id=step_id, status_code=r.status_code, tool_name="serper.search", error_detail=r.text[:200]
        ):
            return []

        if r.status_code >= 400:
            st.status = "api_failure"
            st.meta["run_failure_code"] = "API_FAILURE"
            st.meta["run_failure_provider"] = "serper"
            st.meta["run_failure_detail"] = r.text[:200]
            return []

        organic = (r.json().get("organic") or [])
        out = []
        for it in organic:
            out.append({"title": it.get("title") or "", "link": it.get("link") or "", "snippet": it.get("snippet") or ""})
        return out


class Scraper:
    @staticmethod
    def fetch_text(*, url: str, snippet: str) -> str:
        if not url:
            return snippet or ""
        try:
            with httpx.Client(timeout=18.0, follow_redirects=True) as client:
                r = client.get(url, headers={"User-Agent":"Mozilla/5.0"})
            if r.status_code >= 400:
                return snippet or ""
            html = r.text
            txt = re.sub(r"<(script|style)[^>]*>.*?</\\1>", " ", html, flags=re.S|re.I)
            txt = re.sub(r"<[^>]+>", " ", txt)
            txt = re.sub(r"\\s+", " ", txt).strip()
            return txt[:16000] if txt else (snippet or "")
        except Exception:
            return snippet or ""


def tokenize(text: str) -> List[str]:
    return re.findall(r"[a-zA-Z][a-zA-Z0-9\\+\\#\\.-]{1,}", (text or "").lower())


def cosine(a: Dict[str,int], b: Dict[str,int]) -> float:
    if not a or not b:
        return 0.0
    dot = sum(v * b.get(k,0) for k,v in a.items())
    na = sum(v*v for v in a.values()) ** 0.5
    nb = sum(v*v for v in b.values()) ** 0.5
    if na == 0 or nb == 0:
        return 0.0
    return float(dot/(na*nb))


def recency_hours(snippet: str) -> Optional[float]:
    s = (snippet or "").lower()
    if "today" in s:
        return 6.0
    if "yesterday" in s:
        return 24.0
    m = re.search(r"(\\d+)\\s*hours?\\s*ago", s)
    if m:
        return float(m.group(1))
    m = re.search(r"(\\d+)\\s*days?\\s*ago", s)
    if m:
        return float(m.group(1))*24.0
    return None


def ats_score(resume_text: str) -> float:
    t = (resume_text or "").lower()
    score = 0.0
    if re.search(r"[\\w\\.-]+@[\\w\\.-]+\\.\\w+", t): score += 0.20
    if re.search(r"\\+?\\d[\\d\\-\\s\\(\\)]{8,}\\d", t): score += 0.10
    for h in ["summary","skills","experience","education","projects"]:
        if h in t: score += 0.12
    if "-" in resume_text or "•" in resume_text: score += 0.10
    if len(resume_text) > 1200: score += 0.12
    return max(0.0, min(1.0, score))


def compute_interview_chance(skill_overlap: float, exp_align: float, ats_s: float, market: float) -> float:
    market = max(1.0, float(market))
    score = (0.45*skill_overlap + 0.35*exp_align + 0.20*ats_s) / market
    return max(0.0, min(1.0, float(score)))


def extract_job_skills(job_text: str, resume_skills: List[str]) -> List[str]:
    low = (job_text or "").lower()
    pool = list(dict.fromkeys([*(resume_skills or []), *COMMON_SKILL_DICTIONARY]))
    found = []
    for s in pool:
        s2 = str(s).lower().strip()
        if s2 and s2 in low:
            found.append(s2)
    return list(dict.fromkeys(found))[:40]


class OneClickAutomationEngine:
    """
    Description: Full Career Operating System loop (L0-L9).
    Layer: L0-L9
    """

    def __init__(self) -> None:
        self._s = get_settings()
        self._store = SqliteStateStore()
        self._health = HealthService()
        self._notifier = NotificationService(dry_run=not bool(getattr(self._s, "twilio_account_sid", None)))
        self._sanitize = SanitizeAgent()
        self._parser = ParserAgentService()
        self._parser_eval = ParserEvaluatorService()
        self._learn = LearningResourceService(serper_api_key=getattr(self._s, "serper_api_key", None))

    def _persist(self, st: OrchestrationState) -> None:
        d = _run_dir(st.run_id)
        _save_json(d / "state.json", st.model_dump())
        self._store.upsert_state(run_id=st.run_id, status=st.status, state=st.model_dump(), updated_at_utc=st.updated_at_utc)

    def load(self, run_id: str) -> Optional[Dict[str, Any]]:
        return self._store.get_state(run_id=run_id)

    def start_run(self, *, filename: str, data: bytes, prefs: Dict[str, Any]) -> OrchestrationState:
        st = OrchestrationState.new(env=self._s.environment, mode="agentic", git_sha=None)
        st.meta["preferences"] = prefs
        st.meta.setdefault("job_scores", {})
        st.meta.setdefault("job_components", {})
        st.meta.setdefault("job_meta", {})
        LiveFeed.emit(st, layer="L1", agent="Dashboard", message="Run created. Starting autonomous pipeline…")
        self._persist(st)
        t = threading.Thread(target=self._run, args=(st.run_id, filename, data), daemon=True)
        t.start()
        return st

    def submit_action(self, *, run_id: str, action_type: str, payload: Dict[str, Any]) -> OrchestrationState:
        raw = self.load(run_id)
        if not raw:
            raise ValueError("run_id not found")
        st = OrchestrationState(**raw)
        st.meta["last_user_action"] = {"type": action_type, "payload": payload}
        LiveFeed.emit(st, layer="L5", agent="HITL", message=f"User action received: {action_type}")
        self._persist(st)
        t = threading.Thread(target=self._continue, args=(run_id,), daemon=True)
        t.start()
        return st

    # ---------------- main loop ----------------
    def _run(self, run_id: str, filename: str, data: bytes) -> None:
        raw = self.load(run_id)
        if not raw:
            return
        st = OrchestrationState(**raw)
        run_dir = _run_dir(run_id)
        prefs = st.meta.get("preferences", {}) or {}

        # L2 extract
        st.start_step("l2_extract", layer_id="L2", tool_name="ResumeExtractor", input_ref={"filename": filename})
        LiveFeed.emit(st, layer="L2", agent="ParserAgent", message="Extracting resume text from upload…")
        resume_text = LocalResumeExtractor.extract_text(filename=filename, data=data)
        (run_dir / "resume_raw.txt").write_text(resume_text, encoding="utf-8")
        st.add_artifact("resume_raw", str(run_dir / "resume_raw.txt"), content_type="text/plain")
        st.end_step("l2_extract", status="ok", output_ref={"artifact_key":"resume_raw"}, message="extracted")

        # L0 sanitize
        st.start_step("l0_sanitize", layer_id="L0", tool_name="SanitizeAgent", input_ref={})
        safe = self._sanitize.sanitize_before_llm(
            state=st, step_id="l0_sanitize", tool_name="sanitize_before_llm", user_text=resume_text, context="resume"
        )
        if safe is None:
            st.status = "blocked"
            LiveFeed.emit(st, layer="L0", agent="SanitizeAgent", message="Prompt injection detected. Run blocked.")
            self._persist(st)
            return
        st.end_step("l0_sanitize", status="ok", output_ref={"sanitized": True}, message="pass")
        LiveFeed.emit(st, layer="L0", agent="SanitizeAgent", message="Security check passed.")

        # L2 intake bundle
        st.start_step("l2_parse", layer_id="L2", tool_name="ParserAgentService", input_ref={})
        extracted = self._parser.parse(raw_text=safe, orchestration_state=st, feedback=[])
        _save_json(run_dir / "intake_bundle.json", extracted.to_json_dict())
        st.add_artifact("intake_bundle", str(run_dir / "intake_bundle.json"), content_type="application/json")
        st.end_step("l2_parse", status="ok", output_ref={"artifact_key":"intake_bundle"}, message="parsed")

        # L3 eval (soft)
        ev = self._parser_eval.evaluate(orchestration_state=st, raw_text=safe, extracted=extracted, target_id="resume_main", threshold=float(prefs.get("resume_threshold", 0.55)))
        score = float(getattr(ev, "evaluation_score", 0.0))
        LiveFeed.emit(st, layer="L3", agent="ParserEvaluator", message=f"Intake quality={score:.2f}. Continuing automation.")

        # Do NOT hard stop. Only mark optional cleanup.
        if score < 0.55:
            st.meta["pending_action"] = "resume_cleanup_optional"
            LiveFeed.emit(st, layer="L5", agent="HITL", message="Optional: improve resume text for better ATS + scoring. Pipeline continues.")

        self._persist(st)

        # L3 job hunt across 8 boards for multiple roles
        roles = prefs.get("target_roles") or []
        if isinstance(roles, str):
            roles = [r.strip() for r in roles.splitlines() if r.strip()]
        if not roles:
            roles = [str(prefs.get("target_role","Data Scientist"))]

        location = str(prefs.get("location","United States"))
        remote = bool(prefs.get("remote", True))
        wfo_ok = bool(prefs.get("wfo_ok", True))
        salary = str(prefs.get("salary","")).strip()
        visa_required = bool(prefs.get("visa_sponsorship_required", False))
        recency_limit = float(prefs.get("recency_hours", 36))
        max_jobs = int(prefs.get("max_jobs", 40))
        discovery_threshold = float(prefs.get("discovery_threshold", 0.70))
        max_refinements = int(prefs.get("max_refinements", 3))

        serper_key = getattr(self._s, "serper_api_key", None)
        if not serper_key:
            st.status = "needs_human_approval"
            st.meta["pending_action"] = "missing_serper_key"
            LiveFeed.emit(st, layer="L3", agent="DiscoveryAgent", message="Missing SERPER_API_KEY in .env.")
            self._persist(st)
            return

        discovery = SerperDiscovery(api_key=serper_key, health=self._health)

        resume_skills = [s.lower() for s in (extracted.skills or [])]
        resume_ats = ats_score(safe)
        resume_tokens = {}
        for t in tokenize(safe):
            resume_tokens[t] = resume_tokens.get(t, 0) + 1
        exp_text = " ".join([" ".join(x.bullets) for x in (extracted.experience or [])]) or safe
        exp_tokens = {}
        for t in tokenize(exp_text):
            exp_tokens[t] = exp_tokens.get(t, 0) + 1

        # query builder
        def build_query(role: str) -> str:
            intent = []
            if remote: intent.append("remote")
            if wfo_ok: intent.append("hybrid")
            if not intent: intent.append("on-site")
            salary_part = f'"{salary}"' if salary else ""
            visa_part = '"visa sponsorship" OR h1b OR opt OR cpt' if visa_required else ""
            skill_hint = " ".join(resume_skills[:6])
            return f'{role} {location} {" ".join(intent)} {salary_part} {skill_hint} ({visa_part}) apply'

        # tbs recency (best-effort)
        tbs = "qdr:d" if recency_limit <= 36 else None

        # refinement loop
        query_variants = [build_query(r) for r in roles[:4]]
        all_results: List[Dict[str, Any]] = []

        for attempt in range(max_refinements + 1):
            LiveFeed.emit(st, layer="L3", agent="DiscoveryAgent", message=f"Hunt attempt {attempt+1}: searching 8 boards for {len(query_variants)} roles…")
            seen = set()
            all_results = []
            for qi, q in enumerate(query_variants):
                for b in DEFAULT_JOB_BOARDS:
                    step_id = f"l3_serper_{attempt+1}_{qi}_{b.domain.replace('/','_')}"
                    st.start_step(step_id, layer_id="L3", tool_name="serper.search", input_ref={"role_query": q, "board": b.name})
                    items = discovery.search(st=st, step_id=step_id, query=f"{q} site:{b.domain}", num=10, tbs=tbs)
                    st.end_step(step_id, status="ok", output_ref={"count": len(items)}, message=b.name)
                    for it in items:
                        link = it.get("link") or ""
                        if not link or link in seen:
                            continue
                        seen.add(link)
                        it["board"] = b.name
                        it["role_hint"] = roles[min(qi, len(roles)-1)]
                        all_results.append(it)

            # L4 scrape+score max_jobs
            LiveFeed.emit(st, layer="L4", agent="ScraperAgent", message=f"Scraping + scoring up to {max_jobs} jobs…")
            ranked: List[Dict[str, Any]] = []
            for idx, it in enumerate(all_results[:max_jobs]):
                url = it.get("link") or ""
                snippet = it.get("snippet") or ""
                title = it.get("title") or ""
                board = it.get("board") or "unknown"
                rh = recency_hours(snippet)
                if rh is not None and rh > recency_limit:
                    continue

                step_id = f"l4_scrape_{attempt+1}_{idx+1}"
                st.start_step(step_id, layer_id="L4", tool_name="Scraper", input_ref={"url": url, "board": board})
                text = Scraper.fetch_text(url=url, snippet=snippet)
                st.end_step(step_id, status="ok", output_ref={"chars": len(text)}, message="scraped")

                low = text.lower()
                visa_ok = not any(x in low for x in VISA_NEGATIVE)
                if visa_required and not visa_ok:
                    continue

                job_skills = extract_job_skills(text, resume_skills)
                overlap = len(set(job_skills) & set(resume_skills)) / max(1, len(set(job_skills))) if job_skills else 0.0

                job_tokens = {}
                for t in tokenize(text):
                    job_tokens[t] = job_tokens.get(t, 0) + 1
                exp_align = cosine(exp_tokens, job_tokens)

                market = 1.0
                if "applicants" in snippet.lower():
                    m = re.search(r"(\\d+)\\+?\\s*applicants", snippet.lower())
                    if m:
                        n = int(m.group(1))
                        market = 1.0 + min(1.5, n/200.0)

                score = compute_interview_chance(overlap, exp_align, resume_ats, market)
                if visa_required and any(x in low for x in VISA_POSITIVE):
                    score = min(1.0, score + 0.05)

                missing = [s for s in job_skills if s not in resume_skills][:12]
                jid = url or uuid4().hex

                ranked.append({
                    "job_id": jid,
                    "rank": 0,
                    "role_hint": it.get("role_hint"),
                    "title": title,
                    "board": board,
                    "url": url,
                    "recency_hours": rh,
                    "visa_ok": visa_ok,
                    "matched_skills": list(set(job_skills) & set(resume_skills))[:12],
                    "missing_skills": missing,
                    "components": {
                        "skill_overlap": overlap,
                        "experience_alignment": exp_align,
                        "ats_score": resume_ats,
                        "market_competition_factor": market,
                    },
                    "interview_chance_score": score,
                    "overall_match_percent": round(score*100.0, 2),
                    "rationale": [
                        f"SkillOverlap={overlap:.2f}",
                        f"ExperienceAlignment={exp_align:.2f}",
                        f"ATS={resume_ats:.2f}",
                        f"MarketFactor={market:.2f}",
                        ("VisaOK" if visa_ok else "NoSponsorship"),
                    ],
                })

                st.meta["job_scores"][jid] = float(score)
                st.meta["job_components"][jid] = ranked[-1]["components"]
                st.meta["job_meta"][jid] = {"role_title": title, "company": board, "url": url, "source": board}

            ranked.sort(key=lambda x: float(x["interview_chance_score"]), reverse=True)
            for i, r in enumerate(ranked, start=1):
                r["rank"] = i

            _save_json(run_dir / "ranking.json", ranked)
            st.add_artifact("ranking", str(run_dir / "ranking.json"), content_type="application/json")
            LiveFeed.emit(st, layer="L5", agent="EvaluatorAgent", message=f"Ranked {len(ranked)} jobs. Top={ranked[0]['overall_match_percent'] if ranked else 'n/a'}%")

            # Gate decision
            top_score = float(ranked[0]["interview_chance_score"]) if ranked else 0.0
            if top_score >= discovery_threshold and len(ranked) >= min(20, max_jobs):
                st.status = "needs_human_approval"
                st.meta["pending_action"] = "review_ranking"
                LiveFeed.emit(st, layer="L5", agent="EvaluatorAgent", message="Ranking ready for approval (HITL).")
                self._persist(st)
                return

            if attempt >= max_refinements:
                st.status = "needs_human_approval"
                st.meta["pending_action"] = "low_confidence_discovery"
                LiveFeed.emit(st, layer="L5", agent="EvaluatorAgent", message="Low confidence after retries. Needs guidance.")
                self._persist(st)
                return

            # refine queries (simple deterministic refinement)
            if ranked:
                top = ranked[0]
                hint = " ".join((top.get("matched_skills") or [])[:6])
            else:
                hint = " ".join(resume_skills[:6])
            query_variants = [f"{r} {location} {hint} apply" for r in roles[:4]]
            self._persist(st)

    def _continue(self, run_id: str) -> None:
        raw = self.load(run_id)
        if not raw:
            return
        st = OrchestrationState(**raw)
        run_dir = _run_dir(run_id)
        pending = st.meta.get("pending_action")
        action = (st.meta.get("last_user_action") or {}).get("type")
        payload = (st.meta.get("last_user_action") or {}).get("payload") or {}

        # Optional resume improvement loop
        if pending == "resume_cleanup_optional" and action == "resume_cleanup_submit":
            new_text = str(payload.get("resume_text","")).strip()
            if new_text:
                (run_dir / "resume_raw.txt").write_text(new_text, encoding="utf-8")
                st.add_artifact("resume_raw", str(run_dir / "resume_raw.txt"), content_type="text/plain")
                st.status = "running"
                st.meta["pending_action"] = None
                LiveFeed.emit(st, layer="L2", agent="ParserAgent", message="Resume updated. Restarting run…")
                self._persist(st)
                self._run(run_id, "resume.txt", new_text.encode("utf-8"))
            return

        # Approve ranking -> generate drafts + learning plan for missing skills
        if pending == "review_ranking" and action == "approve_ranking":
            ranking_path = run_dir / "ranking.json"
            intake_path = run_dir / "intake_bundle.json"
            if not (ranking_path.exists() and intake_path.exists()):
                st.status = "needs_human_approval"
                st.meta["pending_action"] = "missing_artifacts"
                self._persist(st)
                return

            ranked = json.loads(ranking_path.read_text(encoding="utf-8"))
            intake = ExtractedResume(**json.loads(intake_path.read_text(encoding="utf-8")))
            top_n = int((st.meta.get("preferences", {}) or {}).get("draft_count", 10))
            ranked = ranked[:top_n]

            # Drafts (ATS resume + cover letter) + learning plan
            drafts: List[Dict[str, Any]] = []
            learning: Dict[str, Any] = {}

            # ATS resume template built from intake (no user ATS required)
            base_resume = self._build_ats_resume(intake)
            (run_dir / "ats_resume_base.md").write_text(base_resume, encoding="utf-8")
            st.add_artifact("ats_resume_base", str(run_dir / "ats_resume_base.md"), content_type="text/markdown")

            for j in ranked:
                jid = j["job_id"]
                title = j.get("title") or "Role"
                company = j.get("board") or "Company"
                missing = j.get("missing_skills") or []

                # Tailored resume (keyword injection + bullets)
                tailored = self._tailor_resume(base_resume, title, company, j.get("matched_skills") or [], missing)
                resume_path = run_dir / f"resume_{jid[:10]}.md"
                resume_path.write_text(tailored, encoding="utf-8")
                st.add_artifact(f"resume_{jid[:10]}", str(resume_path), content_type="text/markdown")

                # Cover letter
                cover = self._cover_letter(intake, title, company, j.get("matched_skills") or [])
                cover_path = run_dir / f"cover_{jid[:10]}.md"
                cover_path.write_text(cover, encoding="utf-8")
                st.add_artifact(f"cover_{jid[:10]}", str(cover_path), content_type="text/markdown")

                # Learning plan for missing skills
                if missing:
                    learning[jid] = self._learn.build_learning_plan(missing_skills=missing)

                drafts.append({
                    "job_id": jid,
                    "title": title,
                    "company": company,
                    "url": j.get("url"),
                    "resume_path": str(resume_path),
                    "cover_path": str(cover_path),
                    "missing_skills": missing,
                })

            _save_json(run_dir / "drafts_bundle.json", {"drafts": drafts, "learning_plan": learning})
            st.add_artifact("drafts_bundle", str(run_dir / "drafts_bundle.json"), content_type="application/json")

            st.status = "needs_human_approval"
            st.meta["pending_action"] = "review_drafts"
            LiveFeed.emit(st, layer="L6", agent="DraftAgent", message=f"Generated {len(drafts)} resume+cover packages + learning plans.")
            self._persist(st)
            return

        # Approve drafts -> mark completed (apply simulation can be added later)
        if pending == "review_drafts" and action == "approve_drafts":
            st.status = "completed"
            st.meta["pending_action"] = None
            LiveFeed.emit(st, layer="L7", agent="ApplyExecutor", message="Drafts approved. (Simulated) submission completed.")
            self._persist(st)
            return

        # Reject ranking -> rerun discovery with refined hint
        if pending == "review_ranking" and action == "reject_ranking":
            reason = str(payload.get("reason","")).strip()
            st.meta.setdefault("user_refinement_notes", [])
            if reason:
                st.meta["user_refinement_notes"].append(reason)
            st.status = "running"
            st.meta["pending_action"] = None
            LiveFeed.emit(st, layer="L5", agent="HITL", message="Ranking rejected. Re-running discovery…")
            self._persist(st)

            resume_path = run_dir / "resume_raw.txt"
            if resume_path.exists():
                self._run(run_id, "resume.txt", resume_path.read_bytes())
            return

        # Reject drafts -> go back to ranking approval
        if pending == "review_drafts" and action == "reject_drafts":
            st.status = "needs_human_approval"
            st.meta["pending_action"] = "review_ranking"
            LiveFeed.emit(st, layer="L6", agent="HITL", message="Drafts rejected. Returning to ranking review.")
            self._persist(st)
            return

        self._persist(st)

    # -------- ATS Resume generation (no user ATS needed) --------
    def _build_ats_resume(self, intake: ExtractedResume) -> str:
        email = intake.contact.email or ""
        phone = intake.contact.phone or ""
        linkedin = intake.contact.linkedin or ""
        github = intake.contact.github or ""
        name = intake.name or "Candidate"

        skills = ", ".join((intake.skills or [])[:25])
        bullets = []
        if intake.experience and intake.experience[0].bullets:
            bullets = intake.experience[0].bullets[:8]

        bullets_md = "\n".join([f"- {b}" for b in bullets]) if bullets else "- Add 4–6 bullets with measurable impact (metrics, scope, tools)."

        return f"""# {name}
{email} | {phone} | {linkedin} | {github}

## Summary
{intake.summary or "AI/ML professional focused on production-grade GenAI and MLOps systems."}

## Skills
{skills}

## Experience
{bullets_md}

## Education
- (Auto-filled from resume intake)
"""

    def _tailor_resume(self, base: str, title: str, company: str, matched: List[str], missing: List[str]) -> str:
        matched_str = ", ".join(matched[:12])
        missing_str = ", ".join(missing[:8])
        return base + f"\n\n## Target Role Alignment\n- Target Role: {title} @ {company}\n- Matched Keywords: {matched_str}\n- Gap Keywords (learning plan attached): {missing_str}\n"

    def _cover_letter(self, intake: ExtractedResume, title: str, company: str, matched: List[str]) -> str:
        name = intake.name or "Candidate"
        email = intake.contact.email or ""
        kw = ", ".join(matched[:8])
        return f"""{name}
{email}

Dear Hiring Manager,

I’m applying for the {title} role. My background includes production-grade AI/ML delivery, GenAI application development, and MLOps practices (CI/CD, reproducible pipelines, evaluation, and governance).

I’m especially aligned to this role through: {kw}.

I’d welcome a quick conversation on how I can help {company} deliver measurable AI impact.

Sincerely,  
{name}
"""


ENGINE = OneClickAutomationEngine()
''')

# Export ENGINE
backup_write("src/careeragent/orchestration/__init__.py", "from .state import OrchestrationState\nfrom .engine import ENGINE, OneClickAutomationEngine\n")

# -------------------------
# app/main.py (bootstrap)
# -------------------------
backup_write("app/main.py", r'''
from __future__ import annotations

import sys
from pathlib import Path

ROOT = Path(__file__).resolve().parents[1]
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))
SRC = ROOT / "src"
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

from app.ui.dashboard import main
main()
''')

# -------------------------
# app/ui/dashboard.py (multi roles + layer panels + HITL)
# -------------------------
backup_write("app/ui/dashboard.py", r'''
from __future__ import annotations

import json
import os
import sys
from pathlib import Path
from typing import Any, Dict, Optional

import requests
import streamlit as st

REPO_ROOT = Path(__file__).resolve().parents[2]
SRC = REPO_ROOT / "src"
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

DEFAULT_API = os.getenv("API_URL", "http://127.0.0.1:8000")


def _safe_json(resp: requests.Response) -> Dict[str, Any]:
    try:
        return resp.json()
    except Exception:
        return {"_raw": resp.text[:1500], "_status_code": resp.status_code}


def _api_get(api: str, path: str, timeout: int = 25) -> requests.Response:
    return requests.get(f"{api}{path}", timeout=timeout)


def _api_post(api: str, path: str, timeout: int = 60, **kwargs) -> requests.Response:
    return requests.post(f"{api}{path}", timeout=timeout, **kwargs)


def _exists(path: Optional[str]) -> bool:
    return bool(path) and Path(path).exists()


def main() -> None:
    st.set_page_config(page_title="CareerAgent-AI Mission Control", layout="wide")
    st.title("CareerAgent-AI — Mission Control (One-Click Automation)")
    st.caption("L0→L9: Intake → Hunt → Scrape/Match → Rank → Draft → Learning Plan → HITL approvals")

    api = st.sidebar.text_input("API Base URL", value=DEFAULT_API)

    # Backend status
    st.sidebar.divider()
    st.sidebar.subheader("Backend")
    try:
        h = _api_get(api, "/health", timeout=3)
        st.sidebar.success("🟢 API Online" if h.status_code == 200 else f"🟠 API issue ({h.status_code})")
    except Exception as e:
        st.sidebar.error("🔴 API Offline")
        st.sidebar.caption(str(e))
        st.stop()

    # Upload
    st.sidebar.divider()
    st.sidebar.subheader("Resume Upload")
    resume_file = st.sidebar.file_uploader("Upload Resume (PDF/TXT/DOCX)", type=["pdf","txt","docx"])

    # Preferences
    st.sidebar.subheader("Targets")
    roles_text = st.sidebar.text_area(
        "Target Roles (1 per line, up to 4)",
        value="Data Scientist\nML Engineer\nGenAI Engineer",
        height=90
    )
    target_roles = [r.strip() for r in roles_text.splitlines() if r.strip()][:4]

    country = st.sidebar.text_input("Country", value="US")
    location = st.sidebar.text_input("Location", value="United States")
    remote = st.sidebar.checkbox("Remote preferred", value=True)
    wfo_ok = st.sidebar.checkbox("On-site/WFO acceptable", value=True)
    salary = st.sidebar.text_input("Salary target (optional)", value="")
    visa_required = st.sidebar.checkbox("Visa sponsorship required (F1/OPT)", value=False)

    st.sidebar.subheader("Job Hunt Controls")
    recency_hours = st.sidebar.slider("Only jobs posted within (hours)", 12, 168, 36, 6)
    max_jobs = st.sidebar.slider("Jobs to score per run", 20, 60, 40, 5)

    st.sidebar.subheader("Quality Gates")
    discovery_threshold = st.sidebar.slider("Min top-score to accept ranking", 0.50, 0.90, 0.70, 0.05)
    max_refinements = st.sidebar.slider("Max query refinements", 1, 6, 3, 1)
    resume_threshold = st.sidebar.slider("Resume quality threshold (soft)", 0.35, 0.90, 0.55, 0.05)
    draft_count = st.sidebar.slider("Draft packages (top jobs)", 3, 20, 10, 1)

    st.sidebar.subheader("Notifications")
    user_phone = st.sidebar.text_input("Phone for SMS (optional)", value="")

    run_btn = st.sidebar.button("🚀 RUN ONE-CLICK", type="primary", use_container_width=True, disabled=(resume_file is None))

    st.sidebar.divider()
    st.sidebar.subheader("Existing Run")
    run_id_in = st.sidebar.text_input("Run ID", value=st.session_state.get("run_id", ""))

    if run_btn:
        prefs = {
            "target_roles": target_roles,
            "country": country.strip() or "US",
            "location": location.strip() or "United States",
            "remote": bool(remote),
            "wfo_ok": bool(wfo_ok),
            "salary": salary.strip(),
            "visa_sponsorship_required": bool(visa_required),
            "recency_hours": float(recency_hours),
            "max_jobs": int(max_jobs),
            "discovery_threshold": float(discovery_threshold),
            "max_refinements": int(max_refinements),
            "resume_threshold": float(resume_threshold),
            "draft_count": int(draft_count),
            "user_phone": user_phone.strip() or None,
        }

        files = {"resume": (resume_file.name, resume_file.getvalue())}
        data = {"preferences_json": json.dumps(prefs)}
        r = _api_post(api, "/analyze", files=files, data=data, timeout=180)
        if r.status_code >= 400:
            st.error(f"/analyze failed: {r.status_code}\n\n{r.text[:1500]}")
            st.stop()
        out = _safe_json(r)
        st.session_state["run_id"] = out["run_id"]
        st.success(f"Run started: {out['run_id']} (status: {out.get('status')})")

    run_id = st.session_state.get("run_id") or run_id_in.strip()
    if not run_id:
        st.info("Upload resume and click RUN, or paste a run_id to monitor.")
        return

    st.button("🔄 Refresh", use_container_width=True)

    r = _api_get(api, f"/status/{run_id}", timeout=25)
    if r.status_code != 200:
        st.warning(f"Run not found yet ({r.status_code}).")
        return
    state = _safe_json(r)

    status = state.get("status","unknown")
    meta = state.get("meta", {}) or {}
    pending = meta.get("pending_action")
    feed = meta.get("live_feed", []) or []
    artifacts = state.get("artifacts", {}) or {}
    evals = state.get("evaluations", []) or []

    c1,c2,c3 = st.columns([1,1,1])
    with c1: st.metric("Run ID", run_id)
    with c2: st.metric("Status", status)
    with c3: st.metric("Pending", str(pending))

    with st.expander("Architecture Layers (L0–L9)"):
        st.markdown("""
- **L0** Security & Guardrails  
- **L1** Mission Control (UI)  
- **L2** Intake Bundle (Profile extraction)  
- **L3** Discovery (8 job boards)  
- **L4** Scrape + Match + Score (20–40 jobs)  
- **L5** Ranking + Evaluator + HITL gates  
- **L6** ATS Resume + Cover Letter generation  
- **L7** Apply (simulated for now)  
- **L8** Tracking (SQLite snapshots)  
- **L9** Learning Plan + Analytics  
        """)

    st.markdown("### Live Agent Feed")
    for ev in feed[-220:]:
        st.write(f"**[{ev.get('layer')} {ev.get('agent')}]** {ev.get('message')}")

    with st.expander("Evaluations (scores + reasons)"):
        if not evals:
            st.caption("No evaluations yet.")
        else:
            for e in evals[-10:]:
                st.write(f"**{e.get('layer_id')} / {e.get('target_id')}** score={e.get('evaluation_score'):.2f} threshold={e.get('threshold')}")
                fb = e.get("feedback") or []
                if fb:
                    st.write("- " + "\n- ".join(fb[:6]))

    # Optional resume improvement
    if pending == "resume_cleanup_optional":
        st.markdown("## Optional Resume Cleanup (improves ATS + scoring)")
        resume_ref = (artifacts.get("resume_raw") or {}).get("path")
        current = ""
        if resume_ref and Path(resume_ref).exists():
            current = Path(resume_ref).read_text(encoding="utf-8", errors="ignore")
        new_text = st.text_area("Paste improved resume text (add email/phone + headings + bullets)", value=current, height=220)
        if st.button("✅ Submit improved resume & rerun"):
            rr = _api_post(api, f"/action/{run_id}", json={"action_type":"resume_cleanup_submit","payload":{"resume_text":new_text}}, timeout=60)
            st.success("Submitted. Refresh in a few seconds.")

    # Ranking
    st.markdown("### Ranking (Top 20–40)")
    ranking_ref = artifacts.get("ranking")
    if ranking_ref and _exists(ranking_ref.get("path")):
        ranking = json.loads(Path(ranking_ref["path"]).read_text(encoding="utf-8"))
        st.dataframe(
            [{
                "rank": x.get("rank"),
                "score_%": x.get("overall_match_percent"),
                "role_hint": x.get("role_hint"),
                "title": x.get("title"),
                "board": x.get("board"),
                "recency_h": x.get("recency_hours"),
                "visa_ok": x.get("visa_ok"),
                "url": x.get("url"),
                "missing_skills": ", ".join((x.get("missing_skills") or [])[:6]),
            } for x in ranking],
            use_container_width=True
        )
    else:
        st.caption("Ranking not available yet.")

    # HITL controls
    st.markdown("### Human-in-the-Loop Controls")
    if status == "needs_human_approval" and pending == "review_ranking":
        c1,c2 = st.columns(2)
        with c1:
            if st.button("✅ Approve Ranking → Generate Draft Packages", type="primary", use_container_width=True):
                _api_post(api, f"/action/{run_id}", json={"action_type":"approve_ranking","payload":{}}, timeout=60)
                st.success("Approved. Generating ATS resume + cover letters + learning plans…")
        with c2:
            reason = st.text_input("Reason (optional) to refine ranking", value="")
            if st.button("❌ Reject Ranking → Re-run Discovery", use_container_width=True):
                _api_post(api, f"/action/{run_id}", json={"action_type":"reject_ranking","payload":{"reason":reason}}, timeout=60)
                st.warning("Rejected. Discovery rerun started.")

    if status == "needs_human_approval" and pending == "review_drafts":
        c1,c2 = st.columns(2)
        with c1:
            if st.button("✅ Approve Drafts → Complete (Simulated Apply)", type="primary", use_container_width=True):
                _api_post(api, f"/action/{run_id}", json={"action_type":"approve_drafts","payload":{}}, timeout=60)
                st.success("Approved. Completing run…")
        with c2:
            reason = st.text_input("Reason (optional) to revise drafts", value="")
            if st.button("❌ Reject Drafts → Back to Ranking", use_container_width=True):
                _api_post(api, f"/action/{run_id}", json={"action_type":"reject_drafts","payload":{"reason":reason}}, timeout=60)
                st.warning("Rejected drafts. Returning to ranking.")

    # Drafts bundle preview
    st.markdown("### Drafts + Learning Plan")
    drafts_ref = artifacts.get("drafts_bundle")
    if drafts_ref and _exists(drafts_ref.get("path")):
        bundle = json.loads(Path(drafts_ref["path"]).read_text(encoding="utf-8"))
        st.json(bundle)
    else:
        st.caption("Draft bundle not available yet.")

    with st.expander("Full State JSON"):
        st.json(state)


if __name__ == "__main__":
    main()
''')

print("\n✅ FULL AUTOMATION PATCH WRITTEN.")
print("Restart backend + frontend.\n")
print("Backend:")
print("  PYTHONPATH=src uv run python -m uvicorn careeragent.api.main:app --host 127.0.0.1 --port 8000 --reload")
print("Frontend:")
print("  API_URL=http://127.0.0.1:8000 PYTHONPATH='.:src' uv run streamlit run app/main.py --server.port 8501")

✅ CWD = /Users/ganeshprasadbhandari/Documents/D_drive/clark/careeragent-ai
BACKUP: /Users/ganeshprasadbhandari/Documents/D_drive/clark/careeragent-ai/src/careeragent/services/db_service.py.bak_20260221_021010
WROTE: /Users/ganeshprasadbhandari/Documents/D_drive/clark/careeragent-ai/src/careeragent/services/db_service.py
BACKUP: /Users/ganeshprasadbhandari/Documents/D_drive/clark/careeragent-ai/src/careeragent/services/health_service.py.bak_20260221_021010
WROTE: /Users/ganeshprasadbhandari/Documents/D_drive/clark/careeragent-ai/src/careeragent/services/health_service.py
WROTE: /Users/ganeshprasadbhandari/Documents/D_drive/clark/careeragent-ai/src/careeragent/services/learning_resource_service.py
BACKUP: /Users/ganeshprasadbhandari/Documents/D_drive/clark/careeragent-ai/src/careeragent/agents/parser_agent_service.py.bak_20260221_021010
WROTE: /Users/ganeshprasadbhandari/Documents/D_drive/clark/careeragent-ai/src/careeragent/agents/parser_agent_service.py
BACKUP: /Users/ganeshprasadbhand